<a href="https://colab.research.google.com/github/ashworks1706/llm-from-scratch/blob/main/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Let's build a Multimodal LLM from scratch!
This isn't about the hype, this is truly for lovers of LLMs and the actual math behind it.

<a href="https://somwrks.notion.site/?source=copy_link" class="btn btn-primary btn-lg" style="background-color: #0366d6; color: white; padding: 5px 10px; border-radius: 5px; text-decoration: none; font-weight: bold; display: inline-block; margin-top: 10px;"><i class="fa fa-file-text-o" aria-hidden="true"></i> Research paper breakdowns</a> <a href="https://github.com/ashworks1706/rlhf-from-scratch" class="btn btn-primary btn-lg" style="background-color: #0366d6; color: white; padding: 5px 10px; border-radius: 5px; text-decoration: none; font-weight: bold; display: inline-block; margin-top: 10px;"><i class="fa fa-file-text-o" aria-hidden="true"></i> RLHF From Scratch</a> <a href="https://github.com/ashworks1706/llm-from-scratch" class="btn btn-primary btn-lg" style="background-color: #0366d6; color: white; padding: 5px 10px; border-radius: 5px; text-decoration: none; font-weight: bold; display: inline-block; margin-top: 10px;"><i class="fa fa-file-text-o" aria-hidden="true"></i> LLM From Scratch</a> <a href="https://github.com/ashworks1706/agents-rag-from-scratch" class="btn btn-primary btn-lg" style="background-color: #0366d6; color: white; padding: 5px 10px; border-radius: 5px; text-decoration: none; font-weight: bold; display: inline-block; margin-top: 10px;"><i class="fa fa-file-text-o" aria-hidden="true"></i> Agents & RAG From Scratch</a>

### But what is a LLM anyway?

A Large Language Model (LLM) is a type of artificial intelligence model designed to understand, generate, and manipulate human language. These models are trained on vast amounts of text data and can perform a wide variety of language tasks such as translation, summarization, question answering, and text generation.

For a more in-depth understanding, check out this explanatory video:

[[ 1 hr Talk ] Intro to Large Language Models by Andrej Karpathy](https://www.youtube.com/embed/zjkBMFhNj_g)

[Large Language Models explained briefly](https://www.youtube.com/watch?v=LPZh9BOjkQs)



<iframe width="560" height="315" src="https://www.youtube.com/embed/zjkBMFhNj_g" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Index

## Preq-requisites

A bit of interest and patience to learning will help.

## Environment Setup

1. Create python environment `python -m venv venv`
2. Install dependencies `pip install -r requirements.txt`
3. Login with our [huggingface account](https://huggingface.co/docs/hub/datasets-usage)
4. Searching for best possible text data given our time, commitment and constraints. We'll use [TinyStories dataset](https://huggingface.co/datasets/roneneldan/TinyStories) for text and [soon] for image-text since it's really high quality and easy to train
5. Here's how I have divided my project structure and pipeline
```
├── models/              # Encoders & fusion layers
├── data/                # Processing scripts
├── training/            # Contrastive loss implementation
├── explainability/      # Attention visualization tools
├── demo/                # Gradio interface
```

In [ ]:
# Upgrade pip to the latest version for compatibility and bug fixes
%pip install --quiet --upgrade pip
# Install all required Python dependencies from requirements.txt
%pip install --quiet -r requirements.txt
# Install ipywidgets for interactive notebook widgets
%pip install ipywidgets


In [ ]:
# Log in to HuggingFace Hub using a token stored in environment variables
# Core Python modules
import os
import math
import glob

# Set PyTorch memory allocation strategy
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# PyTorch core and neural network modules
import torch
import torch.nn as nn

# Data loading and batching
from torch.utils.data import DataLoader

# HuggingFace Hub and datasets
from huggingface_hub import login
from datasets import load_dataset

# Tokenizer (for GPT-2 or similar)
from transformers import GPT2TokenizerFast

# Environment variable management
from dotenv import load_dotenv

import numpy as np

# Load environment variables from a .env file (for secure token storage)
load_dotenv()

# Retrieve the HuggingFace token from environment variables
huggingface_token = os.getenv("HUGGING_FACE_TOKEN")

# Authenticate with HuggingFace, enabling access to datasets and models
login(token=huggingface_token, add_to_git_credential=True)

# Free CUDA memory cache
torch.cuda.empty_cache()
# Enable memory-efficient attention implementation
torch.backends.cuda.enable_mem_efficient_sdp(True)
print(torch.cuda.is_available())

---
## (Optional) Evolution of Large Language Models

Language models have undergone a remarkable evolution over the past few decades. Let's trace this progression to understand how we arrived at today's powerful transformer-based LLMs.

---

### 1. Rule-Based Systems (1950s-1980s)

Rule-based systems emerged in the 1950s and 1960s as the first serious attempt to make computers understand and generate human language. These systems, like ELIZA and SHRDLU, relied on manually crafted rules and pattern-matching logic to simulate conversation or parse commands. At the time, this approach was necessary because computers were extremely limited in memory and processing power, and there was no practical way to “learn” language from data. Instead, engineers and linguists painstakingly encoded linguistic patterns and world knowledge into the system, allowing it to respond to specific phrases or commands in constrained scenarios.

Despite their ingenuity, rule-based systems faced fundamental limitations that ultimately made them impractical for real-world language understanding. Their logic was brittle: they could only handle inputs that matched their predefined rules, and any unexpected phrasing or new vocabulary would cause them to fail. Scaling these systems to handle more complex or open-ended conversations required an explosion of rules, making development and maintenance prohibitively time-consuming and expensive. Furthermore, these systems lacked the ability to learn or adapt on their own; every update or expansion had to be done manually, leaving them unable to keep pace with evolving language or new contexts.

ELIZA demonstrated remarkable effectiveness in specific, constrained scenarios while failing dramatically in open-ended conversations. In therapy-like exchanges, ELIZA's pattern-matching excelled: when a user typed "I am feeling depressed," ELIZA would recognize the "I am [X]" pattern and respond with "Why do you think you are feeling depressed?"-creating an illusion of understanding that was convincing enough to pass a limited Turing test (even outperforming GPT-3.5 in a 2023 study). However, ELIZA collapsed when conversations moved beyond its programmed patterns. For instance, if a user said "My mother-in-law is visiting next week," ELIZA might respond with "Tell me more about our family"-completely missing the contextual nuance and emotional content that might be present. This stark contrast between success in narrow domains and failure in broader contexts highlighted the fundamental limitations of rule-based approaches and ultimately drove the field toward statistical methods that could learn from data.

The legacy of rule-based systems is both instructive and cautionary. While they pioneered key concepts in natural language processing-such as dialogue management and symbolic reasoning-their inflexibility and the combinatorial explosion of rules made it clear that a different approach was needed. This set the stage for the development of statistical and, later, neural language models, which could learn patterns from data and generalize to new inputs without explicit programming. In this way, rule-based systems laid the groundwork for modern AI while also highlighting the need for more flexible, scalable solutions.


### 2. Statistical Revolution (1990s-2010s)  

The statistical revolution emerged as a response to the inflexibility of rule-based systems, driven by the need to handle language’s inherent variability through probabilistic reasoning rather than rigid logic. Researchers shifted focus to algorithms that could *learn* patterns from data, leveraging growing computational power and newly available digital text corpora. This era saw language modeling framed as a probability estimation problem, where systems predicted words based on observed frequencies rather than handcrafted rules.  

N-gram models became the workhorse of this revolution, operating on a simple premise: the probability of a word depends only on the last *N-1* words. For example, a trigram model calculated $ P(wₙ | wₙ₋₁, wₙ₋₂) $ which allowed it to predict "fox" after "the quick brown." These models thrived in applications like early speech recognition, where predicting "phone" after "cellular" improved accuracy in voice-to-text systems. However, their limitations became glaringly apparent at scale. A vocabulary of 20,000 words required 400 million entries for bigrams and 8 trillion for trigrams-numbers that exhausted 1990s storage capabilities. Even with aggressive smoothing techniques, 30% of valid phrases remained unobserved in training data, forcing models to guess blindly.  

Hidden Markov Models (HMMs) attempted to address these issues by introducing latent states for tasks like part-of-speech tagging. In a POS tagger, a "verb" state might emit "running" with 70% probability, while a "noun" state emitted "dog" with 85% likelihood. This worked well for narrow tasks like parsing structured sentences in technical manuals. However, HMMs faltered with ambiguous phrasing-in "Time flies like an arrow," they struggled to disambiguate whether "flies" was a verb or noun because their Markov assumption ignored context beyond the immediate state.  

A telling example of statistical models’ mixed success lies in early email spam filters. N-gram models excelled at flagging messages containing "free money" or "credit card," achieving 95% accuracy on patterned spam. But they failed catastrophically with creative phishing attempts like "our account needs verification" (where "verification" appeared legitimate in other contexts), demonstrating their inability to grasp semantic intent. Similarly, HMM-based speech recognizers transcribed "recognize speech" flawlessly but mangled "wreck a nice beach" into nonsensical outputs, exposing their reliance on surface-level patterns rather than meaning.  

The statistical era ultimately highlighted a fundamental tension: while frequency-based approaches scaled better than rules, they remained shackled to superficial patterns. Models could mimic local syntactic structures but failed to capture the semantic depth or contextual flexibility that defines human language. This realization, coupled with advances in neural network theory, paved the way for architectures that could learn *distributed representations* of meaning-setting the stage for the neural revolution that would follow.


### 3. Neural Dawn (2000s-2017)  

![Transformer attention mechanism](https://github.com/ashworks1706/llm-from-scratch/blob/main/assets/attention.png?raw=1)


The neural dawn marked a paradigm shift from statistical counting to learned representations, driven by advances in computational power and distributed vector spaces. This era bridged the gap between rigid rule-based systems and today’s transformer models, using neural networks to encode language into continuous vectors rather than discrete symbols or counts.  

Word2Vec (2013) revolutionized how machines understood meaning by mapping words to dense 300-dimensional vectors. Unlike n-grams that treated "bank" and "riverbank" as unrelated, Word2Vec learned that these words share semantic neighbors like "water" and "money," enabling analogies like *king - man + woman ≈ queen*. This worked because words appearing in similar contexts (e.g., "deposit" for both financial and geological "banks") were pushed closer in vector space. However, this strength became a weakness with polysemous words: "apple" had the same vector whether referring to fruit or the tech company, forcing models to conflate unrelated meanings. Early search engines used these embeddings to improve query understanding but struggled with niche terms like "GPU acceleration," which appeared too infrequently in training corpora.  

LSTMs/GRUs addressed RNNs’ fatal flaw: the vanishing gradient problem. By introducing gating mechanisms (forget, input, output gates), they could retain critical information over hundreds of steps. For example, in translating "The cat sat on the mat, which was old," LSTMs preserved "cat" across the sentence to correctly gender pronouns in languages like French. But their sequential nature made them slow-processing a 500-word article took 500 sequential steps, creating bottlenecks. A 2016 study found LSTMs achieved 92% accuracy on sentence completion but required 3 seconds per sentence versus transformers’ 0.2 seconds (ACL Anthology). GRUs simplified this by merging gates, trading some long-range memory for faster training, but both architectures remained fundamentally limited by their O(n) time complexity.  

The attention mechanism (2015) was the final breakthrough before transformers. By letting models dynamically focus on relevant context, it solved the "fixed window" problem. In early implementations, an RNN with attention translating "She poured water from the *pitcher* into the *glass*" could attend to "pitcher" when generating "vaso" (glass) in Spanish, even if the words were 20 tokens apart. However, hybrid attention-RNN systems still struggled with computational complexity-each attention step required O(n²) operations for sequence length n. For a 1,000-token document, this meant 1 million operations per layer, limiting real-world usability.  

Legacy: These innovations laid the groundwork for transformers by proving three key principles:  
1. Distributed representations (Word2Vec) could encode semantic relationships  
2. Gating mechanisms (LSTMs) enabled long-term memory  
3. Attention allowed dynamic context focus  

### 4. Transformer Era (2017-Present)  

![OpenAI Transformer](https://github.com/ashworks1706/llm-from-scratch/blob/main/assets/transformer.png?raw=1)


The transformer era began with a radical departure from sequential processing, solving two fundamental limitations of prior architectures: parallelization and context blindness. Unlike RNNs that processed text word-by-word (O(n) time), transformers used self-attention to analyze all tokens simultaneously (O(1) steps). This shift enabled models to handle 10,000-token documents as easily as 10-word sentences, while capturing relationships between distant words like "it" and "antecedent" across paragraphs.

Self-attention became the engine of this revolution. By projecting input tokens into Query (Q), Key (K), and Value (V) vectors, transformers learned dynamic relationships:

This allowed a model translating "The cat didn't eat because it was full" to attend "it" → "cat" for correct pronoun resolution. Positional encodings solved the missing sequence awareness: sinusoidal patterns injected order information without sacrificing parallel processing. For example, in "A→B→C", the positional encoding for B ensures it knows it follows A, even when processing all tokens at once.

The model evolution timeline reveals strategic innovations:

BERT (2018): Masked language modeling enabled bidirectional context understanding. By hiding 15% of tokens (e.g., "The [MASK] sat on the mat"), BERT learned that "cat" fits better than "elephant" through global attention.

GPT-3 (2020): Scaling to 175B parameters unlocked few-shot learning. When prompted with "Translate 'hello' to French:", GPT-3 leverages its vast parameter space to activate relevant language pathways, despite never being explicitly trained on translation.

GPT-4 (2023): Mixture of Experts (MoE) made trillion-parameter models feasible. Different expert subnetworks activate for specific tasks-like one expert handling grammar while another manages factual recall-reducing inference costs by 80% compared to dense models (OpenAI, 2023).

### 5. The Scaling Laws (2020-Present)  
Scaling laws revealed an uncomfortable truth: model performance improves predictably with more parameters, data, and compute, but with diminishing returns and existential costs. The Chinchilla optimality principle showed models like GPT-3 were undertrained: for 175B parameters, they needed 3.5T tokens (20 tokens/parameter) instead of 300B. Retraining with this ratio reduced perplexity by 15% while using 70% fewer parameters (DeepMind, 2022).

Emergent abilities appeared suddenly at scale: 100B parameters: Models gained basic reasoning, solving "If Alice has 3 apples and gives 2 to Bob, she has 1 left", 500B parameters: Code generation emerged, translating Python pseudocode into executable scripts and 1T+ parameters: Multimodal fusion became possible, aligning text with images/video. Hardware advancements enabled this growth explosively: 2017: 8x NVIDIA V100 GPUs → 1B param model (3 weeks training) and 2023: 10,000x H100 GPUs → 1T param model (2 months training). But the energy cost became staggering-training GPT-4 consumed ~50 GWh, equivalent to powering 40,000 homes for a year. This centralization raised ethical concerns, as only tech giants could afford the $100M+ training runs, potentially stifling open research.

The Paradox of Scale: While larger models achieve better benchmarks, their "reasoning" remains pattern matching on steroids. A 2024 study found GPT-4 fails 65% of counterfactual logic puzzles (e.g., "If it rains on Mars, the ground gets wet" → incorrect despite training on Mars data), exposing that scale alone can’t bridge the gap to true understanding. our project’s explainability layer directly addresses this by making the model’s "reasoning" transparent-a critical step toward auditing and improving these systems.

Transformers and scaling laws have created AI systems of unprecedented capability, but at immense computational and environmental costs. The next frontier lies in efficient intelligence-models that match GPT-4’s performance with 100x fewer parameters, perhaps through our project’s multimodal fusion and attention distillation techniques. This era has proven that scale matters, but wisdom lies in using it judiciously.


### 6. Multimodal & Generative Era (2023-Present)  

The current era marks AI's leap beyond pure text into *multisensory intelligence*, where models process and generate images, audio, code, and more alongside language. This shift responds to two critical realizations: 1) Human intelligence is inherently multimodal, and 2) Real-world applications demand systems that mirror this versatility. our project's BERT-ResNet fusion with explainability hooks epitomizes this trend, combining text and vision while exposing decision pathways.  

- Stable Diffusion 3 (2024): Generates 1024px images with legible text in <35 seconds on a 24GB GPU, using a novel Rectified Flow sampler that optimizes the noise-to-image path. Unlike predecessors that produced garbled text (e.g., "Stop" → "Sotp"), SD3's triple encoder architecture (CLIP, OpenCLIP, T5) aligns semantic vectors across modalities.  

<img src="https://github.com/ashworks1706/llm-from-scratch/blob/main/assets/stable-diffusion-transformer.png?raw=1" alt="Stable Diffusion Architecture" width="800"/>

- Multimodal LLMs: Models like GPT-4V and our project's architecture use cross-attention to fuse modalities. For example, when processing "a red apple on a table," our model's BERT encoder extracts "red" and "apple," while ResNet identifies shape/texture, with fusion layers correlating these via:  

<img src="https://github.com/ashworks1706/llm-from-scratch/blob/main/assets/multimodal-transformer.png?raw=1" alt="Multimodal Architecture" width="800"/>


- LLM-Fusion (2023): A materials science breakthrough showing LLMs can unify SMILES notations, text descriptions, and molecular fingerprints, improving property prediction accuracy by 37% over naive concatenation.  

Success Case - Medical Imaging:  
Multimodal systems now diagnose tumors from X-rays + radiologist notes with 92% accuracy (vs. 78% for image-only models). Our project's heatmaps could show which image regions (tumor edges) and terms ("irregular borders") drove the diagnosis, making AI a collaborative tool rather than a black box.  

Failure Case - Context Collapse:  
When asked to generate "a safe beach scene," Stable Diffusion 3 omitted lifeguards due to overzealous NSFW filtering, demonstrating how safety measures can strip context. Similarly, GPT-4V described a protest photo as "a lively crowd" while ignoring central banners, showing modality bias.  

Core Challenges:  
1. Modality Imbalance: Vision often dominates text in training (80% image-text pairs vs 20% text-only), causing models to "hallucinate" visuals from thin textual prompts.  
2. Energy Costs: Training SD3 consumed ~6.5 GWh-equivalent to 500 US homes' annual use-raising sustainability concerns.  
3. Explainability Debt: While attention heatmaps help, interpreting 1.7T parameter models remains akin to "debugging a brain with a flashlight."  

The Road Ahead:  
Next-gen systems like our multimodal LLM prototype aim to balance capability with transparency. By using contrastive loss for alignment and ONNX export for efficiency, we're pioneering *accountable multimodality*-models that don't just see and speak, but show their work. As the field grapples with ethical scaling, our focus on explainability and GPU-efficient training (batch size 64-128) offers a blueprint for sustainable progress.

---

## Step 1 - First Modern Transformer Architecture


This is the core of the project, I want to understand and explain along with you how exactly do LLMs work? what's going on under the hood? why is it like that? how is that possible? the AHA! moments.

At the heart of every modern large language model lies the transformer architecture - a design so revolutionary that it redefined artificial intelligence’s trajectory. While the original 2017 "Attention is All You Need" paper introduced the concept, today’s systems like Gemini, Llama 3, and Gemma have evolved into highly optimized machines for linguistic reasoning. This tutorial will be mainly about the latest architecture while some reflections from the past.  

#### 1. Input Embedding


##### Understand what it means



At the very first stage of a transformer-based Large Language Model (LLM), we encounter the concept of Input Embedding. When a human reads a sentence, we instinctively understand the meaning of each word because our brains associate it with memories, context, and connotations. But machines don’t inherently understand words—they only comprehend numbers. Input embedding is the process of converting human-readable language into a format the model can mathematically operate on. Think of each word or token in a sentence like a unique coordinate in a high-dimensional space—this vector is not just an arbitrary number; it encodes semantic meaning, context, and relational positioning with respect to other words. In modern architectures like Llama 3 or Gemini, these embeddings are often pre-trained so that words like “king” and “queen” end up in similar regions of this multidimensional space, while words like “apple” and “justice” are farther apart. What’s fascinating is that this process is not static; it evolves as part of training, allowing the model to "learn" what semantic space looks like by adjusting these vectors based on billions of training examples.

<img src="https://github.com/ashworks1706/llm-from-scratch/blob/main/assets/tokenization.png?raw=1" alt="Tokenization Visualization" width="1000"/>

But why is input embedding so crucial? Why not just assign each word a number and move on? Let’s say we tried to assign "apple" the number 5 and "banana" the number 10. The model would be tempted to think that "banana" is twice as important or somehow more powerful than "apple," simply because the number is bigger. And if we used one-hot vectors—where each word is represented as a long list of zeros with a single one—it would be a nightmare for the model to infer any relationships, because every word would look equally distant from each other, offering no natural clue about similarity. Instead, by embedding words into a dense vector space using learned parameters, we allow the model to capture nuanced relationships: "Paris" - "France" + "Germany" ≈ "Berlin" is a classic example. This kind of arithmetic over word meanings is only possible because of how embeddings are structured. Moreover, modern LLMs don’t just embed single words—they often embed subwords or byte-pair encodings (BPEs), allowing them to deal gracefully with misspellings, rare words, and new terms that weren’t in the original vocabulary.

At its core, an input embedding is a transformation from a discrete space (a vocabulary of tokens) into a continuous vector space. Suppose we have a vocabulary of size $V$ and we want to embed each token into a d-dimensional space. We initialize an embedding matrix E of shape $V × d$, where each row corresponds to a token’s vector representation. When the model receives an input sequence like “The cat sat,” each token is first converted into a one-hot vector of size $V$, where only one entry is ‘1’ and the rest are ‘0’. Multiplying this one-hot vector by the embedding matrix $E$ effectively selects the corresponding row: $Eᵗ × onehot(token)$ returns the d-dimensional embedding for that token. Now, instead of dealing with sparse binary vectors, the model has access to dense vectors like $[0.23, -1.45, ..., 0.67]$, where each value in the vector can be fine-tuned through gradient descent. During training, as the model receives loss feedback, the embedding matrix $E$ is updated via backpropagation, shifting these vectors to better reflect semantic relationships in the data. Importantly, in most modern LLMs, positional embeddings are then added to these token embeddings, forming a matrix of shape $sequence_length × d$ that becomes the actual input to the transformer.

From a low-level perspective, embeddings are just matrix lookups and additions—but they lay the numerical foundation for all higher-order reasoning that follows. - The dimensionality of this space-often 512 or 4,096 dimensions in modern models-is carefully chosen to balance efficiency and expressiveness. Higher dimensions provide more "room" to capture subtle relationships (e.g., distinguishing "jog" from "sprint" based on intensity), but they also demand more computational resources. Imagine trying to describe a painting: using only three words ("blue," "sad," "ocean") loses nuance, while a 500-word essay captures every brushstroke but becomes unwieldy. Embeddings strike a middle ground, condensing meaning into dense vectors that preserve essential relationships without overwhelming the model. For example, in a 512-dimensional space, the vector for "king" might have high values in dimensions representing royalty, gender, and authority, while "queen" shares the royalty and authority dimensions but differs in gender. This structured yet flexible representation enables the model to perform analogical reasoning, as seen in the classic "king - man + woman = queen" example.

The training process itself resembles a massive collaborative art project. Initially, embeddings are random splatters of paint on a canvas. With each training example (sentence), the model adjusts these splatters slightly-guided by gradient descent-to form coherent shapes. Early iterations produce vague outlines (e.g., clustering animal words), while later stages add fine details (distinguishing "mammals" from "reptiles"). Backpropagation acts as the critical feedback loop, akin to an art instructor pointing out which brushstrokes misrepresent the subject. For example, if the model incorrectly predicts "barked" in "The ___ chased the mailman," backpropagation identifies whether the error stemmed from the embedding for "dog" (too weak) or "mailman" (insufficiently linked to "chase"). Over time, this iterative refinement transforms chaos into a structured semantic landscape, where proximity in the embedding space reflects real-world relationships-not just syntactic similarity but cultural, emotional, and functional associations.

This learning process is why modern LLMs can generalize to unseen words or phrases. When encountering "quokka selfie," the model may not have explicit embeddings for "quokka" (a rare marsupial), but it can infer meaning from subwords ("qua-" associations with "quirky," "-okka" with "marsupial") and context (co-occurrence with "selfie" and "Australia"). This flexibility mirrors how humans handle novel concepts-using partial knowledge to make educated guesses. Without embeddings, models would lack this compositional understanding, treating every word as an isolated entity. The result would be a fragile system incapable of creativity or adaptation, much like a dictionary without example sentences-technically accurate but devoid of contextual nuance.

In summary, input embeddings are the foundational layer where raw text becomes numerical meaning. They encode not just definitions but the rich tapestry of relationships that make language powerful. Through gradient descent and backpropagation, these vectors evolve from random noise into a structured representation of human knowledge-a digital mirror of how we ourselves organize linguistic concepts. This transformation enables everything from basic synonym detection to complex reasoning, forming the bedrock upon which all subsequent transformer layers build. Understanding embeddings isn’t just about grasping matrix multiplications; it’s about appreciating how machines learn to navigate the vast, intricate landscape of human language.

Now imagine if input embeddings didn’t exist. Without them, the model would be forced to rely on hard-coded numerical labels or sparse encodings with no relational structure. The network would be starting from scratch every time it encountered a word, unable to generalize from “walked” to “walking” or “run” to “runner.” The richness of language would be lost, and the model would have to memorize each possible combination rather than learn generalizable patterns. Additionally, training would be massively inefficient—there would be no shortcut for the model to group similar tokens together or understand contextual nuance. The entire semantic landscape that allows a transformer to reason, analogize, and generate would collapse into a shallow, disconnected mess. It's not an exaggeration to say that without high-quality input embeddings, modern LLMs wouldn't just perform worse—they would likely be incapable of functioning in any meaningful sense. The magic would vanish, not because we lost complexity, but because we lost structure.


##### More To Know

- Treat the vocabulary vector space as a huge brain of the computer, the words in it are present in form of vectors, it uses various algorithms to learn the patterns and form relationships between the vocabulary and words it encounters in the space.

    <img src="https://github.com/ashworks1706/llm-from-scratch/blob/main/assets/vector_space.png?raw=1" alt="Vector Space Visualization" width="500"/>

- Gradient descent is the iterative optimization process that enables neural networks, including transformer-based language models, to learn meaningful input embeddings by minimizing prediction errors. At the start of training, embeddings are initialized as random vectors in a high-dimensional space-imagine scattering points randomly across a blank map. These vectors have no inherent semantic meaning, akin to assigning arbitrary coordinates to words like "king" or "apple." The model’s goal is to adjust these coordinates so that geometrically similar vectors correspond to semantically related words. This adjustment happens via gradient descent, which quantifies the "direction" and "magnitude" of changes needed to reduce the model’s errors. For example, if the model mispredicts "prince" instead of "queen" in the sentence "The ___ ruled the kingdom," the loss function calculates how far the current embedding for "queen" is from its ideal position relative to "kingdom" and "ruled." Gradient descent then nudges the embedding vector for "queen" closer to the correct region of the semantic space while pushing "prince" slightly away. The learning rate-a hyperparameter-acts like a step size, controlling how aggressively these adjustments occur. Too large a step risks overshooting optimal positions (divergence), while too small a step prolongs training. Over millions of iterations, gradient descent sculpts the embedding space into a structured landscape where words with related meanings cluster together, and antonyms or unrelated terms drift apart. This process is why "king - man + woman" eventually approximates "queen" in the vector space-- gradient descent aligns these relationships by systematically reducing prediction errors across vast datasets. Without gradient descent, embeddings would remain random, rendering the model incapable of understanding context, synonyms, or analogies-essentially reducing language processing to guesswork.  

    <img src="https://miro.medium.com/v2/resize:fit:800/1*yK6OjDS-1iaPPV0LR6KFjA.png" alt="Gradient Descent Visualization" width="800"/>

- Backpropagation is the mechanism that calculates *how* each embedding vector should be adjusted during gradient descent, tracing the model’s errors backward through its layers to pinpoint responsibility. When the model makes a prediction error-say, outputting "bank" instead of "shore" in the sentence "The boat drifted toward the river ___"-backpropagation determines how much each embedding contributed to this mistake. It starts by computing the gradient of the loss function with respect to the model’s final output, then applies the chain rule of calculus to propagate this gradient backward through the network’s layers. For input embeddings, this means deriving how small changes to each word’s vector would affect the final prediction. In the example, the embedding for "river" might have overly strong associations with "bank" (financial institution) due to training on financial texts, while "shore" was insufficiently linked. Backpropagation quantifies this imbalance, generating gradients that indicate how to adjust "river," "bank," and "shore" in the embedding matrix. Crucially, these gradients are not uniform -- they reflect the unique role each embedding played in the error. For instance, if "drifted" was also in the input, its embedding might receive a smaller gradient if it had weaker connections to the incorrect prediction. This granularity allows the model to refine embeddings contextually-the same word ("bank") might be adjusted differently depending on whether it appears in "river bank" or "investment bank." Over time, these micro-adjustments accumulate, enabling embeddings to capture polysemy and context-dependent meanings. Without backpropagation, the model would lack the precision to attribute errors to specific embeddings, making learning inefficient or impossible. It is the combination of backpropagation’s analytical precision and gradient descent’s iterative refinement that transforms random vectors into a rich, navigable semantic space-the foundation of modern language models’ linguistic prowess.

    <img src="https://media.geeksforgeeks.org/wp-content/uploads/20240217152156/Frame-13.png" alt="Backpropagation Visualization" width="800"/>

- Tokenization is the process of splitting text into units like words or subwords-plays a critical role in shaping embeddings. Consider the word "unhappiness." A word-level tokenizer would treat it as a single token, forcing the model to learn its meaning from scratch. In contrast, a subword tokenizer might split it into "un," "happy," and "ness," allowing the model to leverage pre-existing embeddings for "happy" and common affixes. This approach mirrors how humans decompose unfamiliar words -- a child who knows "happy" and "un-" can infer "unhappy" without being explicitly taught. However, imperfect tokenization can create challenges. For instance, splitting "indivisible" into "in," "di," "vis," and "ible" might obscure its connection to "divide." Modern models mitigate this by training tokenizers and embeddings jointly, ensuring that frequent subwords (like "ing" or "tion") develop consistent semantic associations across words.

    <img src="https://miro.medium.com/v2/resize:fit:1400/1*CdjbU3J5BYuIi-4WbWnKng.png" alt="Tokenization Visualization" width="800"/>




##### Code Implementation

In [ ]:
class InputEmbedding(nn.Module):

    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        # Create an embedding matrix of size (vocab_size, d_model)
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # Scale embeddings by sqrt(d_model)
        return self.embedding(x) * math.sqrt(self.d_model)

- **`nn.Embedding(vocab_size, d_model)`**

   * Internally creates a weight matrix **E** of shape `(vocab_size, d_model)`.
   * When you call `self.embedding(x)`, PyTorch takes each integer token in `x` and does a lookup to grab the corresponding row of **E**, returning a tensor of shape `(batch, seq_len, d_model)`.

- **Scaling by √d\_model**

   * Multiplying by `math.sqrt(self.d_model)` is a trick borrowed from the original Transformer paper: it keeps the variance of the embeddings roughly constant when you move into dot-product attention layers.
   * Without it, if your embeddings have variance σ², your attention scores (which are dot-products of these embeddings) would grow as O(d\_model × σ²), potentially leading to very large or very small gradients.

#### 2. Positional Encoding

##### Understand what it means

When the transformer receives a sequence of token embeddings—say, the vectors corresponding to “The cat sat on the mat”—it sees these embeddings as a bag of vectors. But there's a problem: the transformer, unlike RNNs or LSTMs, has no inherent understanding of the order in which these tokens appear. It treats each input token independently. This is where *Positional Encoding* comes in. It's a mechanism designed to inject information about token order into the model. Without it, the sentence “The cat sat on the mat” would be mathematically indistinguishable from “Mat the on sat cat the.” Positional encoding modifies each token embedding so that the model can differentiate not just *what* the word is, but *where* it is in the sequence. In practice, it augments each word vector with an additional vector that represents its position—like giving each actor in a play not only their lines (token embedding) but also a stage direction (positional encoding). Together, these guide the model in understanding sentence structure and context flow.

But why is positional encoding so critical? Imagine trying to interpret the sentence “Alice gave Bob a book” without knowing the word order—suddenly, the model can’t tell who gave what to whom. That’s because human language is highly dependent on sequential context: in English, word order can determine subject-object relationships, modify meaning through placement of adjectives or adverbs, and structure clauses. The attention mechanism in transformers operates over all token pairs in parallel, and without order, it has no way to distinguish whether a token came earlier or later. Positional encodings fill this gap by giving the model the temporal or sequential cues it otherwise lacks. More subtly, these encodings allow the model to learn dependencies across both short and long ranges. For instance, in the sentence “The report that the CEO signed yesterday was submitted,” the model needs to associate “report” and “was submitted” despite several words of distance. The relative position helps make this link more traceable within the self-attention mechanism.

<img src="https://github.com/ashworks1706/llm-from-scratch/blob/main/assets/positional_encoding_5.png?raw=1" alt="Position Encoding Visualization" width="900"/>


Now let’s talk math. In the original transformer paper, positional encodings were added directly to token embeddings: if $xᵢ$ is the embedding of the $i-th$ token and $pᵢ$ is its positional encoding, the final input becomes $xᵢ + pᵢ$. These encodings $pᵢ$ are not arbitrary—they’re crafted using sine and cosine functions of different frequencies. Specifically, for position $pos$ and dimension $i$, the encoding uses:

$ PE(pos, 2i) = sin(pos / 10000 ^{2i/d})$
$ PE(pos, 2i+1) = cos(pos / 10000 ^ {2i/d})$

This clever trick ensures that each position has a unique signature, and more importantly, that the difference between two positions is preserved in a way the model can learn from. For example, the encoding for position 5 and 6 will have a difference pattern that resembles the one between 10 and 11, giving the model a sense of relative positioning as well. In newer models like T5, Llama, and Gemini, learned positional embeddings are often used instead—these are vectors trained through backpropagation just like token embeddings, and they’re typically added or concatenated with the token embeddings, giving the model greater flexibility in encoding positional dynamics.

What happens if we remove positional encoding altogether? The model becomes blind to order. That is, it sees “He eats quickly” the same way as “Quickly eats he.” For tasks like translation, summarization, or code generation—where token order fundamentally shapes meaning—the model's outputs degrade significantly. Even in seemingly bag-of-words tasks like sentiment analysis, order matters; “not bad” conveys a different sentiment than “bad not.” Without positional encoding, the self-attention layers can’t assign importance to earlier or later tokens differently, leading to a homogenized, chaotic representation where syntactic and semantic relationships get blurred. This breaks the very ability of the model to model grammar, time, nesting of ideas, or causal links. In other words, removing positional encoding is like trying to read a script where every line is shuffled—you might recognize the words, but you’ve lost the story.

##### More to know





- Positional encoding uses sinusoidal functions because they create *unique yet predictable patterns* for each position. Imagine assigning each word in a sentence a musical note: low-frequency sine waves represent broad positional roles (like sentence beginnings), while high-frequency waves capture fine-grained order (specific word positions). This multi-scale approach allows the model to simultaneously recognize that "king" in position 1 and "queen" in position 5 are both nouns (low-frequency similarity) while distinguishing their exact locations (high-frequency difference). The base value 10,000 in the formula acts like a conductor’s tempo control-slowing the "music" for deeper dimensions to prevent harmonic chaos.  

- The genius of sinusoidal encoding lies in how it encodes *relative distances*. For two positions $pos$ and $pos+k$, their encoding vectors share a mathematical relationship: $PE_{pos+k} = PE_{pos} \cdot W_k$ where $ W_k $ is a rotation matrix derived from the frequency terms. This lets the model compute attention scores using vector dot products that inherently reflect positional offsets. It’s akin to two dancers (tokens) coordinating moves based on their stage positions-their movements (attention weights) naturally adapt whether they’re 2 or 20 feet apart.  

- Newer models like T5 use learned positional embeddings, treating positions as learnable parameters. This approach acts like a GPS that customizes maps for each task-unlike the fixed "world map" of sinusoidal encoding. While sinusoidal encodings generalize better to unseen sequence lengths (crucial for zero-shot inference), learned embeddings often achieve higher accuracy on domain-specific tasks by tailoring position rules. For example, in legal document processing, learned embeddings might emphasize section headers’ positions more heavily than body text.  

- The embedding dimension $ d_{model} $ (typically 512-4096) determines how many "languages" of position the model can speak. Each dimension hosts a unique sinusoidal wave, with lower dimensions (e.g., dimension 0) oscillating slowly to mark paragraph-level order, while higher dimensions (e.g., dimension 511) oscillate rapidly to capture word-level adjacency. This multidimensional orchestration prevents positional collisions-ensuring "position 5" in one sentence doesn’t resemble "position 105" in another, even with identical token embeddings.  

- Element-wise addition merges positional and semantic information into a single vector, forcing the model to *jointly* process meaning and location. It’s like dyeing a fabric (token embedding) with patterned ink (positional encoding)-the original material and design become inseparable. This differs from concatenation, which would treat position as separate channels (like wearing a name tag *and* a uniform). Addition’s compactness preserves the transformer’s core efficiency while enabling emergent position-aware semantics.  

- Without positional encoding, the phrase "time flies like an arrow" becomes indistinguishable from "arrow flies like time"-a catastrophic failure for tasks requiring temporal understanding. In machine translation, omitting positional encoding drops BLEU scores by 38% (Google Research, 2023), as subjects and objects randomly swap. Even in bag-of-words tasks like sentiment analysis, "not bad" (positive) and "bad not" (nonsense) become identical, cratering accuracy by 22%. Positional encoding isn’t just helpful-it’s the syntactic glue holding language’s meaning together in transformer models.




##### Code Implementation

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=2048):
        super().__init__()
        # Create a zero matrix to hold positional encodings
        pe = torch.zeros(max_len, d_model)
        # Create a column vector of positions (0 to max_len-1)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # Compute the division term for the sinusoidal frequencies
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # Apply sine to even indices in the embedding dimension
        pe[:, 0::2] = torch.sin(position * div_term)
        # Apply cosine to odd indices in the embedding dimension
        pe[:, 1::2] = torch.cos(position * div_term)
        # Register as a buffer so it is saved with the model but not updated by gradients
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Add positional encoding to input embeddings (broadcasted along batch)
        seq_len = x.size(1)
        return x + self.pe[:seq_len].unsqueeze(0)


The PositionalEncoding class implements a crucial component of transformer architectures that addresses a fundamental limitation: transformers process all input tokens in parallel rather than sequentially, which means they have no inherent understanding of word order. This class creates fixed sinusoidal encodings that inject position information into the model's token embeddings. When initializing, the class first creates an empty tensor (pe) of shape [max_len, d_model] to store position encodings for sequences up to max_len tokens long, where each position is represented by a d_model-dimensional vector. It then generates a sequence of positions from 0 to max_len-1 as a column vector. The heart of the implementation comes next, where it calculates a set of frequency divisors that decrease exponentially across the embedding dimensions using the formula exp(-(log(10000.0) * 2i/d_model)). These divisors create different wavelengths for the sinusoidal functions, with longer wavelengths in earlier dimensions and shorter wavelengths in later dimensions. The class then populates the encoding matrix by applying sine functions to even-indexed dimensions (0, 2, 4...) and cosine functions to odd-indexed dimensions (1, 3, 5...), multiplying each position by the corresponding divisor. This arrangement ensures that each position has a unique encoding pattern while similar positions have similar encodings. The position encoding matrix is registered as a buffer rather than as a trainable parameter, meaning it's saved with the model but not updated during backpropagation. In the forward pass, the method takes an input tensor x containing token embeddings, extracts the sequence length, and simply adds the appropriate slice of positional encodings to these embeddings after broadcasting to match the batch dimension. This addition of positional information enables the self-attention mechanism to recognize and utilize sequence order, allowing transformers to effectively model language despite their parallel processing nature.



#### 3. Multi-Head Self-Attention



##### Understand what it means

The multi-head self-attention mechanism is the heart of the transformer. It's the component that allows every token in a sequence to dynamically interact with every other token and to decide, for itself, which parts of the input are important for understanding or generating the output. In simple terms, self-attention lets a word like "it" in the sentence “The dog chased the ball because it was excited” figure out whether "it" refers to the dog or the ball by comparing and computing relevance scores with every other word. Now, take this idea and multiply it: instead of doing this comparison once, multi-head attention allows the model to run several attention mechanisms in parallel—each with its own set of learned weights and biases. This is crucial, because different heads can specialize in different linguistic phenomena: one head might learn to track subject-verb agreement, another might track coreference resolution, and another might understand temporal cues. After all heads have attended to the input in different ways, their outputs are concatenated and linearly transformed, allowing the model to integrate these multiple views into a unified representation. So in a way, multi-head attention is like having a team of experts, each focusing on a different aspect of language, whose insights are then combined for richer understanding.

<img src="https://github.com/ashworks1706/llm-from-scratch/blob/main/assets/multi-head-self-attention-visualization.png?raw=1" alt="Multi Head Self Attention Visualization" width="900"/>

But why is this mechanism so revolutionary? Before transformers, sequence modeling was dominated by recurrent neural networks (RNNs), which processed tokens one at a time, limiting parallelism and often struggling with long-range dependencies. Self-attention broke this limitation by allowing tokens to consider all other tokens at once. This leads to two major benefits: first, it dramatically improves training efficiency by allowing all tokens to be processed in parallel; second, it gives the model direct pathways to connect distant parts of a sentence or document. For instance, in a legal document, the model might need to connect a clause in the first paragraph to a term defined six pages later. Self-attention creates these connections natively. The “multi-head” aspect amplifies this effect by letting the model explore different dimensions of token relationships at the same time, capturing not just similarity but more abstract relationships like causality, contrast, or syntactic structure. It's also worth noting that attention scores are computed using queries, keys, and values—learned projections of the input that allow the model to fine-tune what aspects of each token are being compared. The design of self-attention as a series of matrix multiplications means that all tokens in a sequence can be processed at once, rather than step by step as in RNNs. This parallelism is not just a theoretical benefit-it’s a practical one, as it allows transformers to take full advantage of the massive parallel computation capabilities of modern GPUs and TPUs. As a result, training and inference can be orders of magnitude faster and more scalable than previous sequential models. Research has shown that different attention heads often specialize in tracking different linguistic or structural patterns, such as coreference, syntax, or positional relationships. Tools for visualizing attention maps can reveal which heads are focusing on which parts of the input, providing valuable insights into how the model is reasoning about language. This interpretability is especially useful for debugging, understanding model failures, and building trust in LLM outputs.

Mathematically, the multi-head attention mechanism begins by projecting each token embedding into three vectors: the $query (Q), key (K), \&\ value (V)$. If the input is a matrix $X$ of shape $(sequence_{length} × d_{model})$, then $Q = XWq$, $K = XWk$, and $V = XWv$, where each $W$ is a learned weight matrix of shape $(d_{model} × d_k)$. The attention score between two tokens is computed as the dot product between the query of one token and the key of another, scaled by $√d_k$ to stabilize gradients: $Attention(Q, K, V) = softmax(QKᵀ / √d_k) V$. This produces a weighted sum of the value vectors, emphasizing those that are most relevant according to the learned attention patterns. For multi-head attention, this process is repeated in parallel across $h$ heads with separate learned projections. The outputs from all heads are concatenated and passed through a final linear transformation: $MultiHead(Q, K, V) = Concat(head₁, ..., head_h)W_o$, where each $headₖ$ is its own attention output. This structure allows the model to attend to multiple semantic and syntactic patterns simultaneously, with each head capturing unique linguistic abstractions. Although we often describe each attention head as having its own set of query, key, and value projection matrices, in practice these projections are implemented as a single large matrix multiplication for efficiency. The result is then reshaped and split into multiple heads, allowing all heads to be computed in parallel as a single batched operation. This not only speeds up computation but also makes the implementation highly optimized for modern hardware like GPUs and TPUs.

After each multi-head attention and feedforward block in the transformer, the model adds the original input back to the output of the block-a technique known as a residual connection. This is followed by layer normalization, which standardizes the activations and helps stabilize training. These two components are critical for training very deep networks, as they help gradients flow backwards through the model and prevent issues like vanishing or exploding gradients, enabling transformers to scale to hundreds of layers without losing performance.

The number of attention heads, the dimensionality of the embeddings, and the size of each head are all hyperparameters that determine the model’s capacity and computational requirements. For example, a model like Llama 2 7B uses 32 attention heads, each with a head dimension chosen so that when all heads are concatenated, they match the overall embedding size. Tuning these values affects not only the model’s ability to capture complex relationships, but also how efficiently it can be trained and run on available hardware.

Attention is used in the Transformer in three places: (1) Self-attention in the Encoder – the input sequence pays attention to itself. (2) Self-attention in the Decoder – the target sequence pays attention to itself. (3) Encoder-Decoder-attention in the Decoder – the target sequence pays attention to the input sequence.

(1) Encoder Self Attention: The input sequence is fed to the Input Embedding and Position Encoding, which produces an encoded representation for each word in the input sequence that captures the meaning and position of each word. This is fed to all three parameters, Query, Key, and Value in the Self-Attention in the first Encoder which then also produces an encoded representation for each word in the input sequence, that now incorporates the attention scores for each word as well. As this passes through all the Encoders in the stack, each Self-Attention module also adds its own attention scores into each word’s representation.

<img src="https://towardsdatascience.com/wp-content/uploads/2021/01/1OYi49Pkg-Vl3D4HleEuu7g.png" alt="Multi Head Attention Visualization" width="900"/>

(2) Decoder Self Attention

Coming to the Decoder stack, the target sequence is fed to the Output Embedding and Position Encoding, which produces an encoded representation for each word in the target sequence that captures the meaning and position of each word. This is fed to all three parameters, Query, Key, and Value in the Self-Attention in the first Decoder which then also produces an encoded representation for each word in the target sequence, which now incorporates the attention scores for each word as well. After passing through the Layer Norm, this is fed to the Query parameter in the Encoder-Decoder Attention in the first Decoder

(3) Encoder Decoder Attention

Along with that, the output of the final Encoder in the stack is passed to the Value and Key parameters in the Encoder-Decoder Attention. The Encoder-Decoder Attention is therefore getting a representation of both the target sequence (from the Decoder Self-Attention) and a representation of the input sequence (from the Encoder stack). It, therefore, produces a representation with the attention scores for each target sequence word that captures the influence of the attention scores from the input sequence as well. As this passes through all the Decoders in the stack, each Self-Attention and each Encoder-Decoder Attention also add their own attention scores into each word’s representation.

When using self-attention in the decoder, especially for generative tasks like text generation or translation, it’s crucial to ensure that the model cannot “see” future tokens when predicting the next word. This is achieved through a technique called causal masking (sometimes called look-ahead masking), where the attention mechanism is restricted so each token can only attend to itself and previous tokens in the sequence, never to tokens that come later. This preserves the autoregressive property needed for generation. Similarly, both encoder and decoder use padding masks to prevent the model from attending to padding tokens, which are inserted to make all sequences in a batch the same length but carry no semantic information. Without these masks, the model could cheat during training or waste capacity on meaningless positions, leading to degraded performance and less reliable outputs.


So, what would go wrong if multi-head self-attention were removed or disabled? At the most basic level, the model would lose its ability to relate tokens across the input sequence. That means no understanding of sentence structure, no resolution of pronouns, no ability to track subjects or parse hierarchical dependencies. Replacing it with a simpler feedforward mechanism would drastically reduce the model’s expressiveness and depth of understanding. Even worse, removing the “multi-head” part would limit the model’s ability to learn diverse patterns—it would have a single narrow view of relationships, unable to attend to multiple interpretations simultaneously. This would severely hurt tasks like translation, question answering, or summarization, where understanding nuance and context from various angles is essential. It's like giving the model one eye instead of many—its field of vision becomes flat, and it misses the richness of meaning that emerges from a fully connected, attention-driven worldview.

##### More to Know



- The multi-head self-attention mechanism operates like a team of linguists analyzing a sentence from multiple theoretical angles simultaneously. Each "head" specializes in distinct linguistic relationships-one might focus on syntactic dependencies (subject-verb-object structures), while another tracks semantic roles (who did what to whom). This division of labor mirrors how human brains process language through parallel neural pathways: visual cortex regions handle word shape recognition, Broca’s area manages grammatical structure, and Wernicke’s area processes meaning. In transformers, the model learns to allocate these specializations organically during training. For instance, when processing "The bank by the river flooded because it rained," one attention head might strongly link "bank" to "river" (resolving geographic context), while another connects "flooded" to "rained" (establishing causality). The magic lies in how these heads evolve without explicit programming-through gradient descent, they discover latent patterns in the data, such as recognizing that "it" in scientific texts often refers to concepts introduced several paragraphs earlier.  

- The query-key-value (QKV) triad forms the computational core of each attention head, functioning like a sophisticated information retrieval system. Imagine a library where: Queries are search terms ("books about neural networks"), Keys are book indexes (matching topics to shelf locations) & Values are the actual book contents. When a token generates a query, it scans all keys in the sequence to find relevant matches, then retrieves blended information from corresponding values. This process allows words to dynamically "pull" context from anywhere in the text-whether three words or three pages prior. The scaling factor $\sqrt{d_k} $ acts as a normalization knob, preventing dot products from ballooning in high-dimensional spaces and ensuring stable gradient flow during training. Without this scaling, attention scores could become dominated by a few dimensions, causing the model to overlook subtle but critical linguistic cues.  

- The softmax operation in attention converts raw similarity scores into probability distributions, essentially forcing the model to make tough choices about where to focus. Think of it as a spotlight operator in a theater: even though every actor (token) is on stage, the light (attention) intensifies on those most relevant to the current scene (context). This selective focus enables the model to handle polysemy-the word "bass" attends strongly to "fish" in one context and "guitar" in another. The multi-head design ensures no single interpretation dominates; like having multiple spotlight operators with different criteria, it captures diverse perspectives simultaneously.  

- Removing multi-head attention would collapse this rich interplay into a monolithic view. A single-head model might correctly resolve "it" in simple sentences but fail spectacularly with nested structures like "The lawyer the judge respected submitted the evidence late." Here, multiple heads collaborate to disentangle the relative clauses: one head links "lawyer" to "submitted," another connects "judge" to "respected," and a third tracks temporal order through "late." Without this collaborative analysis, the model might incorrectly assume the judge submitted the evidence. The multi-head architecture’s true power lies in its ensemble approach-combining weak learners (individual heads) into a strong collective intelligence capable of parsing language’s fractal complexity.  

- This mechanism’s scalability explains why transformers dominate modern AI. While RNNs struggle with sequences longer than 100 tokens due to memory decay, multi-head attention maintains crisp connections across 100,000+ tokens in models like GPT-4. It achieves this through associative memory-storing relationships not in fixed neural states but in dynamically computed attention patterns. Each forward pass essentially "recomputes" the context from scratch, avoiding the memory bloat of recurrent connections. This design choice, radical in 2017, now underpins everything from code completion to protein folding prediction, proving that language’s structure-when analyzed through sufficiently parallel, adaptive lenses-holds keys to general intelligence.



##### Code Implementation

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.d_head = d_model // num_heads
        self.d_model = d_model  # Store d_model as instance variable
        # Linear layer to project input into queries, keys, and values
        self.qkv_proj = nn.Linear(d_model, 3 * d_model)
        # Output projection to combine all heads
        self.out_proj = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None, cache=None, layer_idx=0, seq_pos=0):
        batch, seq_len, _ = x.size()
        # Project input to Q, K, V and split into heads
        qkv = self.qkv_proj(x)
        q, k, v = qkv.chunk(3, dim=-1)
        q = q.view(batch, seq_len, self.num_heads, self.d_head).transpose(1,2)
        k = k.view(batch, seq_len, self.num_heads, self.d_head).transpose(1,2)
        v = v.view(batch, seq_len, self.num_heads, self.d_head).transpose(1,2)

        if cache is not None:
            prev_k, prev_v = cache.get(layer_idx, seq_pos)
            prev_k = prev_k.permute(1,0,2).unsqueeze(0)
            prev_v = prev_v.permute(1,0,2).unsqueeze(0)
            k = torch.cat([prev_k, k], dim=2)
            v = torch.cat([prev_v, v], dim=2)

        # Update cache with new keys and values
        if cache is not None:
            cache.update(layer_idx, k[0,:, -1, :], v[0,:, -1, :], seq_pos + seq_len - 1)

        # Compute scaled dot-product attention scores
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_head)
        # Apply causal mask if needed (prevents attending to future tokens)
        if mask is None:
            if cache is None:
                causal_mask = torch.triu(torch.ones(seq_len, seq_len), 1).bool()
            else:
                causal_mask = torch.zeros(seq_len, seq_len + seq_pos).bool()
                causal_mask[:, :seq_pos] = False
                causal_mask = torch.triu(causal_mask, diagonal=1)
            causal_mask = causal_mask.to(x.device)
            attn_scores = attn_scores.masked_fill(causal_mask, float('-inf'))
            batch_size, seq_len = x.size(0), x.size(1)
            attn_scores = attn_scores.masked_fill(causal_mask.unsqueeze(0).unsqueeze(0), float('-inf'))

        # Convert scores to probabilities with softmax
        attn_weights = torch.softmax(attn_scores, dim=-1)
        # Weighted sum of value vectors
        attn_output = torch.matmul(attn_weights, v)
        # Concatenate heads and project back to d_model
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch, seq_len, self.d_model)  # Use self.d_model
        return self.out_proj(attn_output)

The `MultiHeadSelfAttention` class implements the core attention mechanism that powers modern transformer models, enabling them to process relationships between tokens in a sequence with remarkable effectiveness. This implementation begins with a carefully designed initialization that establishes the architectural foundation for parallel attention computation across multiple representation subspaces. The constructor first validates that the model dimension (`d_model`) is evenly divisible by the number of attention heads through an assertion statement, ensuring that each attention head receives an equal portion of the embedding dimension. This mathematical constraint is fundamental to the transformer architecture, as it enables the clean partitioning of the representation space into `num_heads` independent subspaces, each with dimension `d_head = d_model // num_heads`. The class then creates two critical linear projection layers: first, the `qkv_proj` layer, which efficiently projects input embeddings into queries, keys, and values simultaneously through a single linear transformation (outputting a tensor with three times the model dimension); and second, the `out_proj` layer, which later recombines the processed information from all attention heads back into the original embedding dimension. This parameter-efficient design allows the model to learn different attention patterns across various representation subspaces while maintaining the ability to integrate this information coherently.

The forward pass implements scaled dot-product attention with several sophisticated optimizations for both training and inference scenarios. It begins by projecting the input embeddings into queries, keys, and values through the shared `qkv_proj` layer, then splits this combined projection into three separate tensors using the `chunk` operation along the feature dimension. Each resulting tensor undergoes a crucial reshaping operation that transforms it from shape `[batch, seq_len, d_model]` to `[batch, seq_len, num_heads, d_head]`, followed by a transpose that swaps the sequence length and number of heads dimensions to yield `[batch, num_heads, seq_len, d_head]`. This transformation is essential as it restructures the data to enable parallel processing across all attention heads while keeping the mathematics of the attention mechanism intact. The implementation includes an efficient caching mechanism that stores previously computed keys and values during autoregressive generation, concatenating them with newly computed ones to avoid redundant computations when generating text token by token. This optimization significantly accelerates inference by reducing computational complexity from quadratic to linear with respect to sequence length. The attention scores are computed using matrix multiplication between queries and transposed keys, then scaled by $1/\sqrt{d_{head}}$ to prevent excessive magnitudes in the dot products, which could push the softmax function into regions of extremely small gradients. For autoregressive models, a causal masking mechanism ensures tokens can only attend to previous positions by creating a triangular mask that sets attention scores for future positions to negative infinity, effectively zeroing their probabilities after applying softmax. The masked attention scores are then converted to probability distributions through softmax normalization, producing attention weights that determine how much each input token contributes to each output position. These weights are applied to the value vectors through another matrix multiplication, creating context-aware representations for each token in the sequence. Finally, the attention outputs from all heads are rearranged back to the original sequence order through a transpose operation, concatenated together, and projected back to the original embedding dimension through the output projection layer, completing the multi-head attention mechanism that allows the model to jointly attend to information from different representation subspaces.

#### 4. Feedforward Neural Network (Position-wise FFN)

##### Understand what it means

Once each token has passed through the self-attention layer and received its context-enriched embedding, that representation still needs to be transformed further into a more abstract and useful encoding. This is where the Position-wise Feedforward Neural Network (FFN) comes into play. Think of it like a sculptor refining a rough statue: attention gives you a good shape, a rough figure that includes all the right materials, but the FFN carves out the final features—compressing, amplifying, and distilling information. For each token, independently of others, this feedforward layer applies two linear transformations with a non-linearity in between. Importantly, this operation is done *position-wise*, meaning every token is processed independently with the exact same set of weights—this ensures consistency across the sequence while letting the model learn deeper, richer representations of the token’s meaning. In modern LLMs, this FFN is not just a shallow two-layer MLP anymore; models like Gemini and GPT-4 have large capacity FFNs, sometimes with gated activations, normalization tricks, or even Mixture-of-Experts routing mechanisms that further enhance their dynamic computation per token. This position-wise independence is not only a computational convenience but a deliberate architectural choice that preserves permutation equivariance within the sequence. By applying the same transformation independently to each token, the model maintains the ability to handle variable-length inputs efficiently and prevents unintended information leakage across positions. This clear separation of concerns allows self-attention to focus on inter-token communication, while the FFN specializes in transforming individual token representations.

<img src="https://mukulrathi.com/static/648e5207805f95bf09c330a43d89d295/f207c/neural-net.png" alt="Feedforward Neural Network" width="900"/>


But why is this layer necessary after self-attention already built the context? Self-attention is powerful because it lets tokens learn from one another, but it doesn't necessarily *transform* the internal representation of each token beyond how it relates to others. What’s often needed after attention is a mechanism to nonlinearly mix and project these learned relationships into more expressive spaces. In other words, attention decides *what* to focus on, and the feedforward network helps decide *what to do* with that focus. Imagine you've read a sentence and paid attention to the key points—now your brain needs to make sense of that information: what conclusions to draw, what associations to form, how to condense it. The FFN does that job in the transformer. It enhances or suppresses aspects of each token’s embedding, enables complex transformations beyond linear mappings, and gives the model the power to generalize. It also introduces nonlinearity, crucial for learning complex patterns, which attention mechanisms alone lack since they’re mostly composed of linear operations (except softmax). While the original Transformer used ReLU as the activation function, modern LLMs typically prefer smoother nonlinearities like GELU due to their improved gradient flow and empirical performance. Moreover, some architectures incorporate normalization layers such as LayerNorm or RMSNorm either before or after the FFN, which help stabilize training and improve convergence speed. Another important design choice is the dimensionality of the intermediate layer, often set to be several times larger than the model dimension (commonly 4 to 8 times). This expansion allows the FFN to act as a powerful local feature projector, increasing the model’s capacity to learn complex abstractions without significantly increasing computational cost, as the operations remain parallelizable across tokens.

Mathematically, the FFN applies two learned linear projections separated by a non-linear activation. Given an input token embedding $x \in \mathbb{R}^{d_{\text{model}}}$, the transformation is:
$\text{FFN}(x) = \text{max}(0, xW_1 + b_1)W_2 + b_2$
Here, $W_1 \in \mathbb{R}^{d_{\text{model}} \times d_{\text{ff}}}$ and $W_2 \in \mathbb{R}^{d_{\text{ff}} \times d_{\text{model}}}$ are learned weights, and $b_1, b_2$ are biases. The intermediate dimensionality $d_{\text{ff}}$ is usually several times larger than $d_{\text{model}}$—common values range from 2048 to 8192 in practice. The ReLU or GELU activation introduces nonlinearity, allowing the network to model complex functions. Each token passes through this same FFN independently, so the operation is highly parallelizable, which is a massive benefit during training and inference. In advanced models like PaLM or Gemini, you might also see *Gated Linear Units* (GLUs) or *Swiglu* variants, where one linear transformation is used to gate another via element-wise multiplication: this increases expressiveness and can help with gradient flow in deeper networks.

In practice, the FFN is wrapped within residual connections and normalization layers, forming a standard transformer block structure: input flows through LayerNorm, then self-attention, followed by an Add & Norm step, then the FFN, and finally another Add & Norm. These residual pathways are critical for enabling stable training of very deep transformer stacks by mitigating vanishing gradients and ensuring smooth information flow across layers.

So, what would happen if this feedforward layer were removed or poorly designed? The most immediate impact would be a massive drop in the model’s capacity to learn deep, abstract patterns. Without it, the transformer becomes just a very sophisticated context aggregator, but lacks the machinery to apply transformations to those contexts. It’d be like having eyes that can observe everything around you but no cognitive function to interpret, prioritize, or reshape what’s seen. Especially in deeper models, removing or under-dimensioning the FFN leads to bottlenecks: even if attention captures rich relationships, the model can't evolve its token representations to be more task-appropriate. Removing the nonlinearity would be equally disastrous—it’d collapse the model’s ability to represent complex functions, essentially reducing it to a glorified linear map. Finally, given how these layers stack up across dozens or even hundreds of transformer blocks, eliminating the FFN would mean stripping out a huge portion of the model’s representational power.


##### More to Know






- Before any attention or feedforward operations can take place, each token in the input sequence is mapped to a high-dimensional vector known as an embedding. These embeddings are learned representations that capture semantic and syntactic properties of the tokens, serving as the starting point for all subsequent computations in the transformer.

- The transformer architecture is designed to maximize parallelization and computational efficiency. Unlike recurrent neural networks (RNNs), which process tokens sequentially and depend on previous states, transformers allow every token to be processed independently and simultaneously. This is made possible by the position-wise application of the feedforward network, which applies the same transformation to each token without regard to its neighbors.

- A feedforward neural network, or multilayer perceptron (MLP), is a fundamental building block in deep learning. It consists of one or more layers of linear transformations (matrix multiplications) separated by nonlinear activation functions such as ReLU or GELU. In transformers, the position-wise feedforward network typically has two linear layers with a much larger intermediate dimension than the input or output, enabling the model to learn complex, expressive transformations for each token individually.

- Residual connections and normalization layers, such as LayerNorm or RMSNorm, are essential components of modern deep neural networks, especially in deep transformer architectures. Residual connections allow the model to “skip” certain layers by adding the input of a block to its output, which helps gradients flow backward through the network during training and prevents issues like vanishing gradients. Normalization layers stabilize the distribution of activations, further improving training dynamics and convergence.

- If you are new to neural networks, it is helpful to know that a feedforward neural network is simply a sequence of linear transformations and nonlinearities. This structure allows the network to approximate highly complex functions and is the foundation for most modern deep learning models, including the transformer’s position-wise FFN.

- Gated mechanisms - Instead of standard activation functions like ReLU, models use mechanisms like Gated Linear Units (GLU). These split the input into two parts, apply a sigmoid to one half, and multiply both halves together. This acts like a dynamic "switch" to control information flow, allowing the network to emphasize or suppress specific features.

- Normalization techniques -  Techniques like LayerNorm or RMSNorm are applied before/after the FFN to stabilize training. They ensure consistent input distributions across layers by scaling and shifting activations, preventing extreme values from disrupting gradient flow.

- Mixture of Experts - Instead of one large FFN, MoE uses multiple smaller "expert" networks. A router dynamically selects 1-2 experts per token (e.g., via top-k scoring). Only the chosen experts process the token, reducing computation while increasing model capacity. For example, a token like "quantum" might route to physics/math experts, while "poem" goes to language/stylistic experts


##### Code Implementation

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1, activation='gelu'):
        super().__init__()
        # First linear layer expands dimensionality
        self.linear1 = nn.Linear(d_model, d_ff)
        # Second linear layer projects back to d_model
        self.linear2 = nn.Linear(d_ff, d_model)
        # Add dropout after activation
        self.dropout = nn.Dropout(dropout)
        # Choose activation function (GELU preferred in modern LLMs)
        if activation == 'gelu':
            self.activation = nn.GELU()
        elif activation == 'relu':
            self.activation = nn.ReLU()
        else:
            raise ValueError("Unsupported activation")

    def forward(self, x):
        # Apply first linear, then activation, then dropout, then second linear
        x = self.activation(self.linear1(x))
        x = self.dropout(x)
        return self.linear2(x)

The `FeedForward` class implements a critical component of transformer architectures known as the Position-wise Feed-Forward Network (FFN). This neural network module processes each token representation independently after the self-attention mechanism has captured contextual relationships between tokens. The implementation follows a specific architecture that consists of two linear transformations with a non-linearity in between, exactly as described in the original "Attention is All You Need" paper by Vaswani et al. During initialization, the class creates four key components: two linear projections, a dropout layer, and an activation function. The first linear layer (`self.linear1`) expands the input embeddings from dimension `d_model` to a typically larger intermediate size `d_ff` (often 4x larger than `d_model`), creating a wider representation space where more complex transformations can occur. The second linear layer (`self.linear2`) then projects the transformed representations back down to the original `d_model` dimension, ensuring that the output maintains the same shape as the input for subsequent residual connections. The inclusion of dropout with a configurable rate (defaulting to 0.1) serves as a regularization mechanism that randomly zeros a portion of the neurons during training, improving generalization by preventing co-adaptation of features and reducing overfitting.

Importantly, the class implements flexibility in activation function selection, prioritizing GELU (Gaussian Error Linear Unit) which has become the preferred choice in modern language models due to its smooth gradients and strong empirical performance. GELU approximates $x \cdot \Phi(x)$ where $\Phi(x)$ is the cumulative distribution function of the standard normal distribution, providing a smooth transition between inactive and active states unlike the abrupt change in ReLU. This smoothness helps with gradient flow during backpropagation, particularly in very deep networks. The forward pass executes the core transformation pipeline: first expanding the dimensionality through the initial linear projection, then applying the non-linear activation to introduce complexity and representational power, followed by dropout to improve generalization, and finally projecting back to the original dimension through the second linear layer. In transformer architectures, this feed-forward block is applied independently to each position in the sequence, which is why it's often called "position-wise" - the same transformation is applied to every token's representation regardless of its position in the sequence. This independence preserves the model's ability to process variable-length sequences efficiently. Together with self-attention, this feed-forward network forms the building block of transformer encoder and decoder layers, with each component specializing in different aspects of representation learning: self-attention captures relationships between tokens, while the feed-forward network introduces non-linear transformations that enable the model to approximate complex functions of the contextualized embeddings.

#### 5. Layer Normalization & Residual Connections


##### Understand what it means


Layer normalization and residual connections may not sound as glamorous as attention or embeddings, but they are the silent stabilizers and enhancers that make the towering depths of modern transformers even possible. At their core, these components are about information stability and learning robustness. Residual connections allow information to skip over certain layers, ensuring that core representations from earlier layers are preserved and can still influence the final output. Layer normalization, on the other hand, addresses a problem common to deep neural networks: the internal covariate shift, where inputs to a layer can change distribution over time as parameters update, making learning unstable. Together, these two form the backbone of each transformer block. Every transformer layer typically follows a pattern: apply sublayer (like attention or feedforward), add a residual connection from input to output, then apply layer normalization. In modern implementations like GPT-3.5 and GPT-4, the order is often switched—pre-norm architecture, where normalization happens before the sublayer, is the dominant standard.

But why exactly are these essential, especially in models like Llama or Gemini that can have over 100 transformer layers? Let’s start with residuals. Imagine you’re building a skyscraper. If each floor can only hold weight on its own, the entire building might collapse under the stress of poor design decisions at higher levels. Residual connections are like reinforcement beams that let lower floors continue to support the structure even if higher levels are imperfect. They allow gradients to flow backward during training with less vanishing or exploding, which means models can learn effectively even with immense depth. Now consider normalization. Deep models with billions of parameters tend to suffer from exploding activations or inconsistent internal distributions, which drastically slow learning or lead to divergence. But why is that? Deep neural networks with billions of parameters, like those used in large language models, tend to suffer from exploding activations and inconsistent internal distributions mainly because of how information is processed and passed through so many layers. As data moves through each layer, even tiny changes or errors can get multiplied over and over again, causing the numbers (activations) inside the network to either blow up to extremely large values (exploding activations) or shrink down to almost nothing (vanishing activations). This happens because each layer’s output becomes the next layer’s input, so if the outputs aren’t kept in check, they can quickly spiral out of control. Additionally, as the model trains and its parameters update, the distribution of values going into each layer keeps shifting, which makes it hard for the network to learn consistently-this is called internal covariate shift. When the input to a layer keeps changing unpredictably, the layer has to constantly readjust, which slows down learning and can even cause the whole training process to fail. In very deep models, these problems get worse because there are so many layers for errors to accumulate, making the network unstable and much harder to train. That’s why techniques like normalization and residual connections are essential-they help keep the activations and distributions stable, allowing these giant models to actually learn useful patterns instead of getting lost in chaotic numbers. LayerNorm normalizes across the feature dimensions of a token's embedding, ensuring a stable mean and variance, and helping the optimizer keep updates consistent across time and depth. Without this, especially in long training runs, even a single unstable activation can cause compounding errors across layers. These operations don't just improve performance—they enable training to *work at all* at the largest scales.

<img src="https://i.sstatic.net/E3104.png" alt="Layer Normalization" width="900"/>


LayerNorm operates on an input vector $ x \in \mathbb{R}^d $ (e.g., a token’s embedding) by first computing the mean $ \mu = \frac{1}{d}\sum_{i=1}^d x_i $ and variance $ \sigma^2 = \frac{1}{d}\sum_{i=1}^d (x_i - \mu)^2 $ across all features of the vector. These statistics capture the distribution of the input’s values. The input is then centered and scaled using $ \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} $, where the tiny constant $ \epsilon $ (e.g., $ 10^{-5} $) prevents division by zero. This step ensures the input has a mean of 0 and a variance of 1, which stabilizes the distribution of activations entering the next layer. However, strict normalization could erase important patterns, so learnable parameters $ \gamma $ (a scaling vector) and $ \beta $ (a bias vector) are applied to rescale and shift the normalized values: $ \text{LayerNorm}(x) = \gamma \cdot \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta $. For example, if a feature in $ x $ is critical for the task, $ \gamma $ can amplify it, while $ \beta $ adjusts its baseline level, giving the model flexibility to retain or modify information.  

Residual connections address the vanishing gradient problem in deep networks. In a transformer block, the output of a sublayer (like self-attention or a feedforward network) is added to its original input: $ \text{output} = x + \mathcal{A}(x) $, where $ \mathcal{A}(x) $ represents the sublayer’s transformation. This simple addition ensures the gradient during backpropagation has a direct path to flow through the network without being diluted by nonlinear activations or weight matrices. For instance, in a 100-layer model, gradients for the earliest layers would normally shrink to near-zero due to repeated multiplication by small derivatives (e.g., from activation functions like ReLU). With residuals, the gradient for the loss $ L $ with respect to the input $ x $ becomes $ \frac{\partial L}{\partial x} = \frac{\partial L}{\partial \text{output}} \cdot \left(1 + \frac{\partial \mathcal{A}(x)}{\partial x}\right) $, preserving a baseline gradient even if $ \frac{\partial \mathcal{A}(x)}{\partial x} $ is small.  

In pre-norm architectures (common in modern models like GPT-3), LayerNorm is applied *before* the sublayer: $ \text{output} = x + \mathcal{S}(\text{LayerNorm}(x)) $. This ordering improves gradient flow further. For example, if $ \mathcal{S} $ is a self-attention layer, normalizing $ x $ first ensures the attention mechanism receives inputs with stable statistics, reducing the risk of exploding query/key dot products. Pre-norm also mitigates the "mean shift" problem, where repeated applications of LayerNorm after residuals can cause hidden states to drift in magnitude over many layers. Without these safeguards, activations in a 1-billion-parameter model could grow exponentially (e.g., doubling each layer), leading to numerical instability or saturation of activation functions. Together, LayerNorm and residuals act as a feedback loop: normalization tames erratic activations, while residuals ensure critical signals persist across layers, enabling transformers to scale to unprecedented depths.

If we remove or ignore these two mechanisms, the architecture quickly becomes brittle. Without residuals, the network essentially treats each layer as a replacement for the last, which puts immense pressure on each one to “get things right”—this is both biologically and computationally unrealistic. Gradients would vanish or explode, especially across 50 or more layers. Similarly, without normalization, the model becomes chaotic: values grow uncontrollably, the optimizer starts chasing wildly fluctuating gradients, and the entire system becomes harder to train. It would take longer to converge, be more sensitive to hyperparameters, and might simply fail to learn anything useful at scale. In fact, many early experiments with deep transformers failed until these stabilizers were put in place. If attention is the mind of the transformer, and feedforward networks are its reason, then normalization and residuals are its heartbeat and spine—ensuring balance, structure, and coherence throughout its computational life.



##### More to Know


- Vanishing Gradient occurs when gradients (adjustments made during training) become extremely small in the early layers of deep neural networks, causing those layers to stop learning. This happens because gradients are calculated using the chain rule during backpropagation: each layer’s gradient is a product of derivatives from subsequent layers. In networks with many layers (e.g., transformers), these derivatives-especially from activation functions like sigmoid or tanh-can shrink exponentially. For example, the sigmoid function’s derivative peaks at 0.25, so multiplying gradients across 10 layers reduces them by a factor of $0.25^{10} \approx 10^{-6}$, rendering early layers nearly static. This stalls learning, as weight updates become negligible. Modern solutions include using ReLU (derivative = 1 for positive inputs) or residual connections, which provide direct pathways for gradients to flow backward without attenuation.  

- Internal Covariate Shift refers to unstable changes in the distribution of inputs to neural network layers during training. As parameters in earlier layers update, their outputs (which serve as inputs to later layers) shift in mean and variance. For example, a layer initially receiving inputs centered around 0 might later get inputs skewed to +5, forcing downstream layers to constantly readjust. This instability confuses the optimizer, slows convergence, and can lead to exploding/vanishing gradients. Batch Normalization and LayerNorm address this by standardizing inputs to each layer (zero mean, unit variance) and applying learnable scaling/shifting parameters ($\gamma$, $\beta$). This stabilizes training and allows higher learning rates.  

- Permutation Equivariance describes a model’s ability to produce equivalent outputs even if input tokens are reordered. Transformers achieve this through self-attention and position-wise operations, which treat tokens as a set rather than a rigid sequence. For instance, swapping "cat" and "dog" in "cat chases dog" would still yield the same semantic interpretation. Mathematically, if $f$ is the transformer and $\pi$ is a permutation, $f(\pi(\text{tokens})) = \pi(f(\text{tokens}))$. This property is critical for handling variable-length text, as it ensures the model isn’t overly sensitive to token order while still using positional encodings to inject sequence information. Permutation equivariance allows transformers to generalize across contexts without memorizing fixed positions.

##### Code Implementation

In [ ]:
class PreNormResidual(nn.Module):
    def __init__(self, d_model, sublayer):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)
        self.sublayer = sublayer

    def forward(self, x, *args, **kwargs):
        # Normalize input, apply sublayer, add residual connection
        return x + self.sublayer(self.norm(x), *args, **kwargs)


The `PreNormResidual` class implements a critical architectural pattern found in modern transformer models that combines two fundamental techniques for stabilizing deep neural networks: layer normalization and residual connections. This particular implementation follows the "pre-norm" variant, where normalization is applied before the main computation rather than after it as in the original transformer paper. During initialization, the class receives two key parameters: `d_model`, which specifies the dimensionality of the model's hidden representations (typically ranging from 512 to thousands of dimensions in large language models), and `sublayer`, which is a callable module representing either a multi-head attention mechanism or a position-wise feed-forward network. The normalization component `self.norm` is instantiated as a LayerNorm that operates on the last dimension, standardizing each position's vector by subtracting the mean and dividing by the standard deviation, then applying learned scaling and bias parameters. This normalization helps combat internal covariate shift by ensuring that the input to each sublayer maintains consistent statistical properties throughout training, significantly improving optimization stability in networks with dozens or even hundreds of layers.

The forward pass encapsulates the elegant simplicity of the pre-norm residual design: first, the input `x` is normalized via `self.norm(x)`, creating a clean, standardized representation; then this normalized input is processed through the sublayer (which could be attention or a feed-forward network); finally, the original unnormalized input is added back through the residual connection `x + ...`. This residual pathway is paramount for deep transformers as it creates a direct gradient highway during backpropagation, mitigating the vanishing gradient problem and allowing information to flow unimpeded across many layers. The inclusion of arbitrary `*args` and `**kwargs` in the forward method provides essential flexibility, allowing the class to transparently pass additional parameters (such as attention masks, key-value caches for generation, or layer indices) to the sublayer without needing to modify its interface. The pre-norm approach has emerged as the preferred configuration in many recent transformer variants because it further improves training stability compared to the post-norm approach, particularly for deeper models. By normalizing before computation, the model ensures that sublayers always receive inputs with consistent statistical properties, making the optimization landscape smoother and allowing for larger learning rates and faster convergence. This seemingly simple wrapper around transformer sublayers thus embodies years of architectural refinements that have enabled transformers to scale to unprecedented depths and capabilities, forming the backbone of modern language models by maintaining stable gradient flow and consistent representation statistics throughout their immense depth.

#### 6. Output Projection & Language Modeling Head

##### Understand what it means

At the tail end of a transformer, after a token passes through several stacked attention and feedforward layers, we’re left with a vector representation—often called a “hidden state”—that encodes the model’s understanding of the token in context. But raw vectors aren't directly useful for producing language. So, the final step is to transform these high-dimensional internal states into probabilities over the vocabulary space, effectively predicting “what comes next.” This transformation is called the output projection layer, and it’s a linear mapping from the hidden state dimension (say, 6144 in GPT-4) to the size of the vocabulary (which might be 50,000+ tokens). Once projected, this output is passed through a softmax function, converting the raw logits into probabilities, and the token with the highest probability is selected (or sampled with temperature, top-k, or nucleus methods during inference). In most decoder-only LLMs like LLaMA, GPT, and Gemini, this head is *tied* with the embedding matrix used at the input layer, meaning the same matrix is used both to embed tokens and to decode them—reducing parameters and reinforcing shared structure between input and output spaces.

<img src="https://jalammar.github.io/images/t/transformer_self-attention_visualization_3.png" width="400">

But why is this final projection and softmax setup so essential? Think of the model’s internal layers as a rich thought process, like a writer brainstorming ideas. But until the writer picks a word to put on the page, none of those thoughts matter. The output projection is the final act of writing—a decision to express something in language. It has to compress all contextual information about past tokens, positional patterns, syntactic relationships, and even world knowledge into a single probability distribution. The softmax ensures that the model doesn't just throw out a single guess—it quantifies *confidence* over all vocabulary items, which is vital for beam search, top-k sampling, or controlled generation. The tying of input embeddings and output projection is also more than just parameter sharing; it enforces a symmetry where what the model “understands” is aligned with what it can “say,” improving sample efficiency and fluency. Imagine if humans had separate vocabularies for understanding and speaking—it would be inefficient and prone to errors. This shared structure helps language models be coherent and grounded in their own internal semantics.

Let’s dive into the math. Suppose the transformer outputs a hidden state vector $h \in \mathbb{R}^d$ for a particular token position. The output projection is a matrix $W \in \mathbb{R}^{V \times d}$, where $V$ is the vocabulary size. We compute the logits $z \in \mathbb{R}^V$ as: $z = W h$

Each component $z_i$ corresponds to the unnormalized score for the $i$-th vocabulary token. These logits are then passed through the softmax function: $\text{softmax}(z)_i = \frac{e^{z_i}}{\sum_{j=1}^{V} e^{z_j}}$

This yields a probability distribution over the entire vocabulary. The model can now use these probabilities to select the next token. If weight tying is used, then the embedding matrix $E \in \mathbb{R}^{V \times d}$ from the input stage is reused, so $W = E$. This means that the direction in embedding space that a hidden state points to is effectively being decoded into the most similar vocabulary word—anchoring the model’s output to what it has learned from input tokens.

If this component were removed or mishandled, the entire language model would break down at the final moment. Without output projection, there's no mechanism for converting thoughts into speech. You’d have an intelligent mind trapped in silence. More technically, without this projection layer, the model cannot map from internal representations to discrete token predictions—so it can’t be trained to predict the next word, nor can it generate coherent output during inference. Without softmax, the model’s outputs would be unbounded and uninterpretable. And if the embedding and projection weights aren't tied in small or medium models, the model has to learn two separate representations of language—making learning slower and more error-prone. It's like a musician learning to read music with one instrument and perform it on a completely different one. Overall, the output projection and language modeling head are not just the last layer—they’re the final translator between machine thought and human language.



##### More to Know

- The softmax function is absolutely essential at the final stage of a transformer model because it takes the raw output scores (logits) from the model’s last linear layer and converts them into a meaningful probability distribution over the entire vocabulary. Mathematically, for each possible next token, softmax computes $\text{softmax}(z)i = \frac{e^{z_i}}{\sum{j=1}^{V} e^{z_j}}$, where $z_i$ is the logit for the $i$-th token and $V$ is the vocabulary size. This operation not only ensures that all probabilities are positive and sum to one, but also accentuates the differences between the most likely and least likely tokens, making the model’s predictions more decisive. Without softmax, the model’s outputs would be uninterpretable raw numbers, and there would be no principled way to select or sample the next token. Softmax is also crucial for training: it enables the use of cross-entropy loss, which directly compares the predicted probability distribution to the true next-token label. In practice, softmax is what allows a transformer to turn its internal, high-dimensional “thoughts” into actionable, human-readable language, quantifying its confidence in every possible word it might generate next.



##### Code Implementation

In [ ]:
class OutputProjection(nn.Module):
    def __init__(self, d_model, vocab_size, embedding_weight=None):
        super().__init__()
        # Linear layer mapping from d_model to vocab_size
        self.proj = nn.Linear(d_model, vocab_size, bias=False)
        # Optionally tie weights with input embedding matrix
        if embedding_weight is not None:
            self.proj.weight = embedding_weight

    def forward(self, x):
        # Output logits for each token position
        return self.proj(x)



The `OutputProjection` class implements the final crucial transformation in a transformer-based language model, serving as the bridge between the model's high-dimensional internal representations and human-readable vocabulary. After a sequence passes through multiple transformer blocks, each token is represented by a dense vector of dimension `d_model` (typically ranging from 512 to several thousand dimensions in modern architectures), which encapsulates rich contextual information about the token's meaning and relationships. The output projection layer transforms these intricate representations into logit scores across the entire vocabulary space through a straightforward yet computationally significant linear projection. During initialization, the class creates a bias-free linear transformation (`nn.Linear`) that maps from the model's hidden dimension (`d_model`) to the vocabulary size, essentially creating a matrix of shape `(vocab_size, d_model)` where each row can be interpreted as a learned representation of a vocabulary token in the model's embedding space. The forward method performs this projection by multiplying the input tensor `x` of shape `(batch, seq_len, d_model)` with the weight matrix, yielding raw, unnormalized logit scores of shape `(batch, seq_len, vocab_size)` that indicate the model's preference for each possible next token at each position in the sequence.

A particularly elegant optimization implemented in this class is weight tying, where the output projection's weight matrix shares parameters with the input embedding layer. When the optional `embedding_weight` parameter is provided, the projection layer's weights are directly assigned to reference this shared parameter tensor, effectively forcing the model to use the same transformation matrix for both embedding tokens at the input and projecting hidden states back to vocabulary space at the output. This technique serves dual purposes: it significantly reduces the model's parameter count (eliminating approximately 25% of parameters in many architectures) and enforces a beneficial symmetry between how tokens are represented when they are read (input embedding) versus when they are generated (output projection). The design deliberately omits bias terms in the linear projection, as including them would break the elegant symmetry of weight tying - the input embedding typically doesn't include a bias vector, so adding one to the output projection would create an asymmetry in how tokens are represented in the input versus output spaces. Research has shown that this parameter-efficient design choice not only reduces model size but often improves performance, as the shared representation creates a more cohesive learned vocabulary space where the meanings of tokens remain consistent whether they're being understood (embedded) or generated (projected). After this projection, the resulting logits are typically passed to a softmax function (implemented elsewhere) to convert them into a probability distribution over the vocabulary, from which the next token can be sampled during text generation.

#### 7. Loss Function & Backpropagation


##### Understand what it means


The loss function is the core feedback mechanism for any neural network, and for large language models, it's typically the cross-entropy loss. After the model has predicted a probability distribution over vocabulary tokens via softmax, we need to compare that distribution to the actual correct token—the ground truth. Cross-entropy measures how far apart these two distributions are. Think of it like this: you’re playing a game where the model tries to guess the next word in a sentence, and the loss function is the scorekeeper telling the model how “bad” its guess was. A perfect guess leads to zero loss; an entirely wrong guess leads to a high loss. This loss is then used to adjust every single parameter in the model to minimize future mistakes. In effect, the loss function translates the model’s errors into numerical signals that flow backwards through the network, adjusting each weight to better capture patterns in language.

<img src="https://editor.analyticsvidhya.com/uploads/18870backprop2.png" width="600">

But why is this specific loss function—cross-entropy—used, and what makes backpropagation so powerful? Cross-entropy is especially suited to classification tasks like language modeling, where the goal is to predict a single correct output from many possibilities. It doesn’t just punish the model for guessing wrong—it punishes it more severely the *more confidently* it guesses wrong. This subtlety is crucial. If your model thinks "pineapple" is definitely the next word, but the answer is actually "president," cross-entropy gives it a strong signal to correct course. This is essential for learning fine-grained differences between similar-sounding or related tokens. Backpropagation, on the other hand, is the process of taking that loss and “walking it back” through the model’s layers, using the chain rule of calculus to determine how each weight contributed to the error. Imagine you’re coaching a soccer team: not only do you need to tell the team they lost the match (the loss), but you also need to analyze each player’s move to figure out who made mistakes and how to fix them. Backpropagation does exactly this—assigning credit or blame to each neuron’s activation and updating parameters via gradient descent to improve the model’s future performance.

Let’s now see this through the lens of math. Say the model predicted logits $z \in \mathbb{R}^V$, and softmax transformed it into probabilities $p = \text{softmax}(z)$. Let the true token index be $y$, so the ground-truth distribution is a one-hot vector with a 1 at index $y$.

The cross-entropy loss $L$ is:$L = -\log(p_y) = -\log\left(\frac{e^{z_y}}{\sum_{j=1}^{V} e^{z_j}}\right)$

This loss gets minimized using gradient descent. Backpropagation computes the derivative of the loss with respect to each model parameter (say, a weight $w$), using the chain rule:

$\frac{\partial L}{\partial w} = \frac{\partial L}{\partial z_y} \cdot \frac{\partial z_y}{\partial w}$

This process is recursively applied through every layer—attention, feedforward, normalization, etc.—adjusting each weight matrix and bias term so that next time the same token context arises, the model makes a slightly better guess. These gradients are then fed into an optimizer like Adam, which scales and smooths them, and applies the update step using learning rates, momentum, and adaptive factors to ensure convergence. Without this step, there is no learning—just a fixed, static transformer generating nonsense.

If we were to remove or misconfigure the loss function or backpropagation, the model wouldn’t learn. Period. No amount of architectural complexity could compensate for the lack of a training signal. It would be like building a highly advanced robot and then never turning it on. Or worse, feeding it incorrect feedback—training it to think wrong answers are right. A poorly chosen loss function could lead the model to optimize for irrelevant goals (like maximizing confidence over correctness), and without backpropagation, there’s no way to update weights based on performance. Additionally, without gradient flow through layers, earlier layers—those responsible for foundational understanding—would become stale and disconnected. The model’s knowledge would be shallow, brittle, and biased toward recent data. This is why training stability, gradient clipping, and optimizer configuration are as crucial as the transformer layers themselves. Learning in LLMs isn’t just about building intelligence—it’s about teaching it through structured, rigorous feedback loops.


##### More to Know

- In machine learning, the loss function quantifies how far the model's predictions are from the actual target values. For language models, this is typically the cross-entropy loss, which measures the difference between the predicted probability distribution (from softmax) and the true distribution (usually a one-hot vector for the correct token). The lower the loss, the better the model's predictions match the ground truth.

- This loss is well-suited for classification tasks like language modeling because it not only penalizes wrong predictions, but penalizes them more if the model is highly confident and still incorrect. Mathematically, for a true token index $y$ and predicted probabilities $p$, the cross-entropy loss is $L = -\log(p_y)$, where $p_y$ is the predicted probability for the correct token.

- Backpropagation is the algorithm that allows neural networks to learn from their mistakes. After computing the loss, backpropagation calculates the gradient (rate of change) of the loss with respect to every model parameter (weight and bias) using the chain rule from calculus. This tells the model how to adjust each parameter to reduce future loss.

- Once gradients are computed, an optimizer such as stochastic gradient descent (SGD) or Adam updates the model’s parameters in the direction that most reduces the loss. This process is repeated over many training examples, gradually improving the model’s performance.

- Training involves a forward pass (input goes through the network to produce a prediction and compute loss) and a backward pass (backpropagation computes gradients, and the optimizer updates weights).

- Without a loss function, the model would have no way to measure its errors or improve. Without backpropagation, there would be no mechanism for the model to learn which weights to adjust and by how much. These mechanisms are fundamental to all deep learning, enabling large language models to learn complex patterns in language and continually improve with more data and training.

- Think of the loss function as a scorekeeper in a game, telling the model how well it did after each round. Backpropagation is like a coach reviewing the game, figuring out which moves led to mistakes, and training the players (model parameters) to do better next time. Gradient descent is the process of making those adjustments, step by step, until the team improves.

##### Code Implementation

In [ ]:
# Tokenizer setup for GPT-2 with a custom pad token
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Computes cross-entropy loss between model logits and target token IDs
def compute_loss(logits, targets):
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
    # Flatten logits and targets for loss computation
    logits = logits.view(-1, logits.size(-1))
    targets = targets.view(-1)
    return loss_fn(logits, targets)


The code establishes the foundational components for processing text and computing loss in a transformer-based language model. It begins by initializing a GPT-2 tokenizer with a crucial modification for batch processing. The `GPT2TokenizerFast.from_pretrained("gpt2")` statement loads a pre-trained, optimized tokenizer that implements byte-pair encoding (BPE), a subword tokenization strategy that effectively balances vocabulary size and token granularity by breaking words into meaningful subunits. This tokenizer enables the conversion between human-readable text and the numerical representation required by neural networks. Since the original GPT-2 tokenizer lacks a padding token (as GPT-2 was primarily trained with continuous text), the code explicitly adds a custom '[PAD]' token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`. This addition is essential for batch processing, where sequences of different lengths must be padded to a uniform length to form rectangular tensors for efficient parallel computation.

The `compute_loss` function implements the training objective for the language model using cross-entropy loss, which measures the difference between the model's predicted probability distribution over the vocabulary and the ground truth. The function begins by initializing `nn.CrossEntropyLoss` with the `ignore_index` parameter set to the padding token's ID, ensuring that padding tokens don't contribute to the loss calculation or gradient updates—a crucial optimization that prevents the model from wasting capacity learning to predict meaningless padding tokens. The function then performs a critical tensor reshaping operation: it flattens the three-dimensional logits tensor of shape (batch_size, sequence_length, vocabulary_size) to a two-dimensional tensor of shape (batch_size×sequence_length, vocabulary_size), and similarly reshapes the target tensor from (batch_size, sequence_length) to (batch_size×sequence_length). This transformation effectively treats each token position as an independent classification problem, allowing the loss function to compute the cross-entropy between predicted and actual token distributions across all positions simultaneously. The reshaping operation preserves the mathematical relationship between predictions and targets while adapting the tensors to the format expected by PyTorch's cross-entropy implementation. When this loss is backpropagated, it guides the model to assign higher probabilities to correct next tokens at each position in the sequence, ultimately teaching the model to generate coherent and contextually appropriate text.

#### 8. Data, Batching, Tokenization, and Parallelism


##### Understand what it means

At the foundation of any large language model lies a massive corpus of text, and the way we handle this data—cleaning, tokenizing, batching, and distributing it—is just as important as the model architecture itself. Training infrastructure refers to everything that supports the learning process: the pipelines that feed data into the model, the hardware setups that run computation in parallel, and the optimization tricks that make training feasible and stable. It starts with data preprocessing, where massive unstructured corpora like Common Crawl, Wikipedia, books, and proprietary sources are cleaned, deduplicated, and filtered for quality. These raw texts are then converted into tokens, which are the actual input units that a model sees. Unlike words or characters, tokens are often subword units generated using Byte-Pair Encoding (BPE) or SentencePiece. The choice of tokenizer greatly affects how efficiently the model learns and how well it generalizes, especially in multilingual or domain-specific settings. Once we have tokens, they're arranged into batches—chunks of data processed together for computational efficiency—and distributed across multiple GPUs or TPUs using strategies like data parallelism, tensor parallelism, and pipeline parallelism. But why all this complexity? Because training an LLM isn’t like training a small model on a laptop—it’s like trying to teach a language to a continent-sized brain. We’re dealing with hundreds of billions of parameters and trillions of tokens. Without batching, we’d waste compute cycles processing tokens one at a time, and without parallelism, we’d never finish training in a human lifetime. Tokenization is equally crucial: it ensures that the model doesn't have to learn from scratch how to represent every possible word in every possible language. Subword tokenization compresses rare words into manageable units, allowing the model to generalize better. Batching boosts efficiency and statistical stability—by averaging gradients across multiple examples, it smooths learning and prevents overfitting to individual samples. And parallelism? That’s the secret sauce that makes it all scale. Tensor parallelism slices up the model’s internal computations across devices, pipeline parallelism sequences layers across GPUs, and data parallelism trains multiple models on different data shards, synchronizing updates via collective communication. The synergy of these systems allows models like Gemini or GPT-4 to train on petabytes of data with hundreds of billions of parameters.

<img src="https://miro.medium.com/v2/resize:fit:900/0*CKEc4j27kiRRJFJ-.jpg" width="800">

Mathematically, tokenization converts a raw text string into a sequence of integers. For instance, with BPE, common substrings are merged greedily based on frequency to build a vocabulary. Then a text like “unhappiness” might be tokenized into [“un”, “happi”, “ness”], which are then mapped to their vocab indices like [1093, 4721, 887]. Each token becomes an input to the embedding layer. For batching, suppose you have a batch size of 4 and a context window of 512 tokens: you’re feeding in a 4×512 matrix per forward pass. In terms of parallelism, imagine splitting your 175B parameter model across 8 GPUs. Tensor parallelism might shard the matrix multiplications inside attention or feedforward layers across GPUs, while pipeline parallelism assigns different transformer blocks to different devices. At each training step, gradients are computed for the loss, gathered from all parallel devices, and the optimizer applies updates in a synchronized fashion using operations like all-reduce. Without efficient sharding and scheduling here, you’d run into out-of-memory errors or inefficient GPU usage. If this infrastructure were missing or poorly implemented, the entire training pipeline would collapse. Training a large model without batching would be so slow that you'd never reach convergence before the hardware dies. Using a bad tokenizer could inflate the vocabulary size or fragment semantically important units, making it harder for the model to learn linguistic structure. Imagine the word “unhappiness” always being split into 6 characters instead of 3 meaningful chunks—learning long-range dependencies becomes harder and slower. Likewise, without parallelism, you couldn’t train modern models on commodity GPUs. You’d hit memory ceilings, slow throughput, and face weeks-long training runs. Poor data handling could lead to overfitting on noisy or duplicated samples. Training infrastructure isn’t just the scaffolding—it’s the skeleton and circulatory system of the LLM. Without it, there is no deep learning, only shallow failure.



##### More to Know


- Data Collection and Preprocessing: The foundation of any LLM is a vast and diverse dataset. This data is collected from sources like web crawls, books, Wikipedia, and proprietary corpora. Before training, the data undergoes rigorous cleaning and normalization-removing duplicates, filtering out low-quality or irrelevant content, and standardizing formats. Techniques such as URL filtering, deduplication, and normalization (e.g., min-max scaling, z-score standardization) are used to ensure the resulting dataset is both high-quality and consistent, which is crucial for effective learning and faster convergence.

- Tokenization: Tokenization is the process of breaking raw text into smaller units called tokens, which can be words, subwords, or even characters. Modern LLMs typically use subword tokenization schemes like Byte-Pair Encoding (BPE) or SentencePiece, which allow the model to efficiently represent rare and compound words by splitting them into common subword units. Each token is mapped to a unique integer ID that the model can process. Popular tokenization libraries include Hugging Face Tokenizers, SentencePiece, and OpenAI’s Tiktoken, all of which are optimized for speed and flexibility across different languages and domains.

- Batching: Batch size refers to the number of training samples processed together in one forward and backward pass. Larger batch sizes can speed up training by making better use of hardware and providing more stable gradient estimates, but they require more memory and can lead to out-of-memory errors if set too high. If GPU memory is limited, gradient accumulation can be used: smaller batches are processed sequentially, and gradients are summed before updating model weights, effectively simulating a larger batch size. Choosing the right batch size is a balance between hardware constraints and training stability.

- Parallelism: Training LLMs with billions of parameters is only feasible through parallelism-splitting the workload across multiple GPUs or TPUs. There are several strategies: Data Parallelism: Each device trains on a different chunk of data with its own model copy, synchronizing gradients after each step, Tensor Parallelism: Large matrix operations within a model are divided across devices, allowing each device to handle a slice of the computation, Pipeline Parallelism: Different layers or blocks of the model are assigned to different devices, and data is passed through them sequentially.

- Why It Matters: Efficient data handling, smart tokenization, optimal batching, and robust parallelism are not just technical details-they are the backbone of scalable LLM training. Without them, models would train too slowly, run out of memory, or fail to generalize. Poor tokenization could fragment important concepts, while bad batching or parallelism could waste computational resources or lead to unstable learning. Modern LLMs rely on these infrastructure choices as much as on their neural architectures to achieve state-of-the-art results.

##### Code Implementation

In [ ]:
# Loads and preprocesses the TinyStories dataset for training
raw_ds = load_dataset("roneneldan/TinyStories", split="train")
print(f"Total examples in dataset: {len(raw_ds)}")


def preprocess_fn(example):
    # Tokenize text with shorter max_length
    enc = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256  # Reduced from 512
    )
    enc["labels"] = enc["input_ids"].copy()
    return enc

# Apply preprocessing to the dataset and format for PyTorch
tok_ds = raw_ds.map(
    preprocess_fn,
    batched=True,  # Process in batches for speed
    num_proc=4,    # Use multiple CPU cores
    remove_columns=["text"]
)
tok_ds.set_format("torch", columns=["input_ids", "labels"])

# Collate function to assemble input and target batches
def collate_fn(batch):
    inputs = torch.stack([item["input_ids"] for item in batch])
    targets = torch.stack([item["labels"] for item in batch])
    return inputs, targets

# Set device for training (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


The provided code implements a comprehensive data preparation pipeline for training a language model on the TinyStories dataset. This process begins by loading the dataset using Hugging Face's `load_dataset` function, which retrieves the TinyStories corpus - a collection of simple stories specifically created for training small language models. The pipeline then prints the total number of examples available in the dataset to provide context about the training corpus size. The heart of this preparation pipeline is the `preprocess_fn` function, which transforms raw text examples into model-ready token sequences. This function tokenizes each text example with parameters carefully selected to balance computational efficiency and model performance: it enables truncation to handle examples exceeding the maximum length, applies padding to ensure uniform sequence lengths (essential for batch processing), and sets a maximum length of 256 tokens (reduced from 512 to optimize memory usage while preserving sufficient context). Crucially, the function creates a "labels" field that is an exact copy of the "input_ids", which establishes the self-supervised learning paradigm where the model learns to predict each token based on previous ones - the fundamental mechanism behind autoregressive language modeling. The dataset transformation continues with the `map` method, which applies the preprocessing function across all examples, leveraging performance optimizations through batch processing and parallel execution across 4 CPU cores. The `remove_columns=["text"]` parameter eliminates the original text data to conserve memory since it's no longer needed after tokenization. After preprocessing, `set_format("torch", columns=["input_ids", "labels"])` converts the tokenized data into PyTorch tensors specifically for the input IDs and labels, making them immediately compatible with PyTorch models without further conversion steps during training. To facilitate efficient batch processing during model training, the code defines a `collate_fn` function that stacks individual examples into batched tensors with dimensions (batch_size, sequence_length). This function takes a list of tokenized examples and returns properly formatted input and target tensors that can be fed directly into the model's forward pass. Finally, the code determines the appropriate computation device (GPU if available, otherwise CPU) to optimize training performance based on the hardware environment. This entire data preparation workflow demonstrates the essential steps required to transform raw text data into the structured, batched tensor format necessary for efficient training of neural language models, with careful attention to performance optimizations like parallelization, memory management through reduced sequence lengths, and hardware acceleration.

#### 9. Interference Optimization & Caching

##### Understand what it means


Inference refers to the process of running a trained language model to generate outputs—completions, answers, summaries—based on given inputs. While training is an expensive one-time process, inference happens continuously and must be fast, efficient, and scalable. Every time you ask ChatGPT a question or an LLM processes a document in a search engine, that’s inference in action. But inference is deceptively expensive. You’re often running hundreds of transformer layers on thousands of tokens across billions of parameters. Modern LLMs like Gemini or GPT-4 don’t recompute every single step from scratch for every token—they rely on inference-time optimizations. This includes attention caching, where intermediate computations like key and value vectors are stored and reused instead of recalculated; quantization, where models are compressed into lower-precision formats (like 8-bit or 4-bit) to reduce memory and increase throughput; and KV caching, where transformer outputs from previous tokens are reused during autoregressive decoding. Combined with efficient batch scheduling and model serving infrastructure (like vLLM or TensorRT-LLM), these systems can reduce inference latency from seconds to milliseconds—crucial for real-time applications.

But why do these optimizations matter so deeply? Let’s say a user inputs a prompt of 1000 tokens and wants a 500-token response. A naive implementation would recompute all layers from scratch for each new output token—a total of 1000 + 1, then 1000 + 2, and so on up to 1500 forward passes. That quickly becomes prohibitively expensive, especially for models with 100+ layers. Caching lets us remember the key and value tensors computed for each token, so when we generate token #501, we don't recompute tokens 1–500. Instead, we *reuse* their embeddings and attention outputs, and compute only what’s new. This dramatically cuts down computation and latency. Quantization, on the other hand, is like storing your clothes in vacuum-sealed bags: smaller, lighter, easier to move. Using 8-bit integers instead of 16- or 32-bit floats doesn’t just reduce memory—it also increases the number of operations that can fit in cache, improving GPU throughput. These techniques allow companies to serve millions of LLM queries per day at a fraction of the cost. Some setups even use speculative decoding or distillation-based acceleration, where smaller models predict probable outputs ahead of the larger model’s verification.


<img src="https://r4j4n.github.io/blogs/img/kv/kv.png" width="800">

Mathematically, let’s look at attention caching. In a transformer layer, each token's output depends on all previous tokens through self-attention. At inference time, you compute the query (Q), key (K), and value (V) matrices from the input embeddings. Normally, for each new token, you would recompute the entire QKV process for the whole sequence. But in caching, you compute Q for the current token, and reuse previously computed K and V tensors for the past context. So instead of recomputing a full matrix-matrix product every time, you compute only a single row of the query matrix and perform a matrix-vector dot product against the cached keys and values. This slashes complexity from quadratic to linear with respect to sequence length at generation time. Additionally, quantization maps 32-bit floating-point weights to 8-bit integers using min-max scaling or learned ranges, effectively performing approximate matrix multiplications that retain most of the model’s performance while using less bandwidth and memory.

<img src="https://www.researchgate.net/publication/369449854/figure/fig1/AS:11431281129400844@1679541267815/Overall-quantization-process-of-Q-HyViT-on-the-representative-structure-of-hybrid-vision.png" src="Quatization of Transformers" width="800">

Quantization in transformers is a powerful technique that allows large language models to run faster and use less memory by reducing the numerical precision of their weights and activations. Instead of storing and computing with standard 32-bit floating-point numbers, quantization maps these values to lower-precision formats, such as 8-bit or even 4-bit integers. This is achieved by scaling and rounding the original weights, often using a simple formula like $W_q = \text{round}\left(\frac{W - \mu}{s}\right)$, where $W$ is the original weight, $\mu$ is a zero-point offset, and $s$ is a scaling factor. The result is that the model’s parameters take up much less space in memory and can be processed much more efficiently by modern hardware, especially GPUs and specialized AI accelerators that are optimized for integer operations. There are several approaches to quantization: post-training quantization, which converts a finished model to lower precision without retraining; quantization-aware training, which simulates quantization during training to help the model adapt; and hybrid methods that mix different precisions for different parts of the model. While quantization can sometimes lead to a small drop in accuracy-especially at very low bit-widths-careful calibration and modern toolkits have made it possible to quantize even very large transformers with minimal performance loss. This makes it feasible to deploy models like Llama or GPT-4 on commodity hardware, edge devices, or in high-throughput cloud environments, dramatically reducing the cost and energy required for inference.

If we didn’t use inference optimization techniques, LLMs would be borderline unusable in production. Latency would be too high for interactive experiences like chatbots or real-time translation. Without caching, a 10-token response might take 10x longer than it should. Without quantization, models might not fit on commodity hardware, inflating deployment costs and energy usage. Think of it like trying to run a high-speed train using a bicycle engine—you’d burn out the system. Worse, the lack of batching or serving optimization would mean your servers can only handle one or two users at a time. That’s not scalable. You'd also encounter memory fragmentation, slower throughput, and even numerical instability if precision isn’t handled carefully. In summary, inference optimization and caching aren’t luxuries—they’re necessities. They’re the engineering glue that transforms theoretical models into useful, accessible, lightning-fast assistants.


##### Contextual Reference

- Inference Optimization: Inference is the process of running a trained language model to generate outputs from new inputs. Unlike training, which is performed once and can be slow, inference must be extremely fast and efficient because it happens every time a user interacts with the model. Optimizing inference is crucial for deploying large language models in real-world applications, where latency and cost are major concerns.

- Attention and KV Caching: During autoregressive generation (such as text completion), each new token depends on all previous tokens. Instead of recalculating all attention computations for every step, modern LLMs use key-value (KV) caching. This technique stores the key and value matrices computed for each token, so when generating new tokens, the model only needs to compute attention for the latest token and reuse previously cached values. This reduces the computational complexity from quadratic to linear with respect to sequence length, dramatically speeding up inference and lowering resource consumption.

- Quantization: Quantization compresses model weights and activations from high-precision floating-point numbers (like 32-bit floats) to lower-precision formats (such as 8-bit or even 4-bit integers). This reduces memory usage, allows models to fit on smaller or cheaper hardware, and speeds up matrix operations by enabling more efficient use of hardware caches and specialized instructions. Quantization can be applied post-training or during model deployment, and with careful calibration, it preserves most of the model’s accuracy while providing significant speedups and cost savings.

- Serving and Batching: Efficient inference also depends on how requests are handled and scheduled. Techniques like request batching (processing multiple user queries together) and model caching (keeping frequently used models or data in fast-access memory) help maximize throughput and minimize latency. Advanced serving frameworks and hardware accelerators (like GPUs, TPUs, or specialized inference chips) are often used to further scale up performance and handle millions of requests per day.

- Why These Matter: Without these optimizations, inference with large models would be prohibitively slow, expensive, and resource-intensive. Caching and quantization allow for real-time responses and make it feasible to deploy LLMs on commodity hardware or in cloud environments. In production, these techniques are not optional-they are essential for making large language models practical, scalable, and accessible to users at scale.

- Recent Innovations: Research continues to improve inference efficiency, introducing new memory caching architectures (such as differentiable memory caches), dynamic and adaptive inference strategies, and hybrid quantization-compression methods. These advances further reduce latency and cost, enabling even larger and more capable models to be deployed in real-world settings.

##### Code Implementation

In [ ]:
class KVCache:
    def __init__(self, num_layers, max_seq_len, num_heads, head_dim):
        # For each layer, store zero-initialized key and value tensors
        self.cache = [{
            "key": torch.zeros(max_seq_len, num_heads, head_dim),
            "value": torch.zeros(max_seq_len, num_heads, head_dim)
        } for _ in range(num_layers)]

    def update(self, layer_idx, new_key, new_value, seq_pos):
        # Update cache at the given layer and sequence position
        self.cache[layer_idx]["key"][seq_pos] = new_key
        self.cache[layer_idx]["value"][seq_pos] = new_value

    def get(self, layer_idx, seq_len):
        # Retrieve cached keys and values up to seq_len for a layer
        return (
            self.cache[layer_idx]["key"][:seq_len],
            self.cache[layer_idx]["value"][:seq_len]
        )


The `KVCache` class implements a critical optimization technique for transformer-based language models that dramatically improves inference efficiency during autoregressive text generation. This class creates and manages a memory structure that stores the key and value tensors computed during the attention mechanism's forward pass, eliminating redundant computations when generating text token-by-token. In the initialization method, the class creates a list of dictionaries with length equal to the number of transformer layers in the model (`num_layers`). Each dictionary contains two zero-initialized tensors—"key" and "value"—with dimensions carefully designed to accommodate the model's architecture: `max_seq_len` representing the maximum possible sequence length, `num_heads` for the number of attention heads, and `head_dim` for the dimension of each attention head. This pre-allocation strategy reserves memory upfront to avoid costly resizing operations during generation. The `update` method provides a mechanism to store newly computed key and value vectors at a specific position in the sequence (`seq_pos`) for a particular transformer layer (`layer_idx`), effectively building up the cache incrementally as each new token is processed. Meanwhile, the `get` method retrieves all cached key and value tensors up to a specified sequence length for a given layer, returning them as a tuple that can be directly used in subsequent attention computations. This caching mechanism is particularly valuable during autoregressive generation because without it, each new token would require recomputing attention patterns for all previous tokens, resulting in quadratic time complexity as sequence length grows. With KV caching, the model only needs to compute new key-value pairs for the most recently generated token and can reuse all previously computed values, reducing the complexity from $O(n^2)$ to $O(n)$ where $n$ is the sequence length. This optimization makes real-time text generation feasible even for long outputs, as the computational cost of generating each additional token remains relatively constant rather than increasing with sequence length. The cache structure is designed to be memory-efficient while still providing rapid access to the exact tensor slices needed during the attention computation, striking a balance between performance and resource utilization.

#### 10. Post-Processing & Output Formatting

##### Understand what it means

This is the often-overlooked stage that takes the generated tokens and prepares them for real-world use. Think of this like a chef plating a dish after it’s been cooked—sure, the taste is what matters most, but presentation, cleanliness, and readiness for consumption are what ultimately make it usable. In the context of an LLM, this step ensures that the raw output (a sequence of tokens) is properly detokenized, aligned with the expected structure, formatted for APIs, escaped for JSON safety, or even enriched with metadata. It also includes optional operations like trimming whitespace, handling tokenization artifacts (like unwanted Unicode sequences), and formatting for specific output modalities—like turning the answer into valid XML, Markdown, or structured JSON, depending on the context.

But why is post-processing even necessary if the model already gave us “an answer”? The key insight is that the LLM itself only works with tokens—it doesn’t know whether the output will be fed into a webpage, an API, a CLI, or a robot. It just produces a sequence of likely tokens. But for an actual application, there’s often a **target schema** or expected format: a JSON blob, a SQL query, a clean sentence, a full Markdown document, a LaTeX block, etc. This is where post-processing plays its magic. If the output is going into a chatbot, we might wrap certain parts in HTML tags, or bold the assistant’s messages. If the output is going into a code interpreter, we’ll want to extract only the code block and strip everything else. Post-processing is also where safety and alignment filters may run—checking if the output includes toxic language, or contains hallucinated URLs, or violates specific rules. It's also the layer where longer outputs might be chunked into paragraphs, tables might be reconstructed, or URLs converted to clickable links. So while the model may have generated something intelligible, post-processing makes sure it’s *usable*—and that it aligns with the product’s expectations, safely and correctly.

From a mathematical standpoint, this stage is less about algebra and more about formal language and parsing. Still, it includes deterministic processes based on rules, regular expressions, and tree-walking algorithms. For example, if the model is prompted to return JSON, we might use a formal parser to validate that the brackets are matched, the keys are quoted, and no special characters break the syntax. Token detokenization itself, which turns token IDs back into strings, is a precise mapping defined by the tokenizer’s vocabulary. For Byte-Pair Encoding (BPE) or SentencePiece tokenizers, detokenization involves joining subwords while applying any necessary decoding logic, such as reversing special tokens or merging byte-encoded artifacts. In structured output scenarios, such as LangChain or JSON mode in OpenAI API, post-processing might involve validating against a JSON schema using tools like `jsonschema`, or even transforming the model’s output via regexes or small grammars to ensure well-formedness. If alignment is enforced at this layer, you might apply penalties or reject completions based on statistical classifiers or safety layers that scan for PII, offensive content, or hallucinated facts. The whole point here is precision and cleanup: no math magic like attention or gradients, but a different kind of magic—cleanliness and reliability.

If this stage were omitted or improperly handled, you would immediately see problems in production. Outputs might have garbled Unicode sequences (like weird � characters), malformed code (syntax errors), or broken structures (incomplete JSON or HTML). In chatbot systems, skipping this stage might mean the user sees raw token artifacts like `<|endoftext|>` or broken Markdown that doesn't render. In structured tasks like SQL generation, unvalidated outputs could crash systems or inject vulnerabilities (yes, post-processing is part of *security* too). You also lose opportunities to enforce consistency—e.g., if a tool like ReAct requires the output to follow a format like `Thought:` → `Action:` → `Observation:`, failure to enforce that format post-generation can lead to breakdowns in tool-use chains. Moreover, skipping safety filters here would leave you vulnerable to edge-case completions that may contain inappropriate content, hallucinated citations, or biased language—especially when the model seems confident but is wrong. In short, post-processing is what ensures the bridge between LLM generation and real-world deployment is robust, clean, safe, and consistent.

##### Code Implementation


In [ ]:
def quantize(tensor, bits=8):
    """Basic linear quantization"""
    # Compute scaling factor based on max absolute value
    scale = tensor.abs().max() / (2**(bits-1)-1)
    # Quantize tensor to int8 range
    qtensor = torch.clamp((tensor/scale).round(), -2**(bits-1), 2**(bits-1)-1).to(torch.int8)
    return qtensor, scale

def dequantize(qtensor, scale):
    # Restore quantized tensor to approximate original values
    return qtensor * scale

# Extends KVCache to store quantized keys and values for memory efficiency
class QuantizedKVCache(KVCache):
    def __init__(self, num_layers, max_seq_len, num_heads, head_dim, bits=8):
        super().__init__(num_layers, max_seq_len, num_heads, head_dim)
        self.bits = bits
        # Store scaling factors for dequantization (per layer, for key and value)
        self.scales = torch.zeros(num_layers, 2, device='cpu')

    def update(self, layer_idx, new_key, new_value, seq_pos):
        # Quantize new key and value tensors before storing
        q_key, scale_k = quantize(new_key, bits=self.bits)
        q_val, scale_v = quantize(new_value, bits=self.bits)
        self.cache[layer_idx]["key"][seq_pos]   = q_key
        self.cache[layer_idx]["value"][seq_pos] = q_val
        self.scales[layer_idx, 0] = scale_k
        self.scales[layer_idx, 1] = scale_v

    def get(self, layer_idx, seq_len):
        # Dequantize cached keys and values for use in attention computation
        q_keys = self.cache[layer_idx]["key"][:seq_len]
        q_vals = self.cache[layer_idx]["value"][:seq_len]
        scale_k, scale_v = self.scales[layer_idx]
        return dequantize(q_keys.float(), scale_k), dequantize(q_vals.float(), scale_v)


The code implements a memory-efficient quantization system for transformer models that significantly reduces the memory footprint during inference while maintaining acceptable accuracy. The quantization process begins with the `quantize()` function, which performs linear quantization by first calculating a scaling factor based on the maximum absolute value in the input tensor. This scaling factor is computed by dividing the maximum value by $(2^{bits-1}-1)$, where $bits$ typically equals 8, corresponding to the maximum positive value representable in 8-bit signed integers (127). After determining the scale, the function divides the original tensor by this scale and rounds the result to the nearest integer, effectively mapping the continuous floating-point values to discrete integers. To prevent overflow, these values are then clamped to the range $[-2^{bits-1}, 2^{bits-1}-1]$ and converted to the memory-efficient `int8` data type. Complementing this, the `dequantize()` function reverses this process by simply multiplying the quantized tensor by the saved scaling factor, restoring the values to approximate their original magnitudes while introducing minimal quantization error. The `QuantizedKVCache` class extends the base `KVCache` mechanism (which normally stores key-value pairs in transformer attention) with this quantization capability. It maintains the same interface as the original cache but adds internal quantization, storing each attention key and value vector as 8-bit integers rather than 32-bit floating-point values—a 4x memory reduction. During initialization, it creates a small tensor of scaling factors (two per layer: one for keys, one for values) to enable accurate dequantization later. The `update()` method quantizes incoming key-value vectors before storing them in the cache, preserving their scaling factors, while the `get()` method retrieves the quantized values, converts them back to floating-point, and applies the appropriate scaling to recover the approximate original values. This approach is particularly valuable during autoregressive text generation, where KV caching is critical for performance but can consume enormous amounts of memory as sequence length grows. By introducing a small amount of precision loss through quantization (typically imperceptible in language model outputs), this implementation achieves dramatic memory savings that enable longer context windows and more efficient inference on memory-constrained devices. The entire system works transparently with the rest of the transformer architecture, requiring no changes to the core attention mechanism or training process, making it an elegant optimization for deployment scenarios.

#### Introduction to Finetuning (Optional)

If the base model is the engine of a car, fine-tuning is like calibrating that engine for specific terrain—off-road, racing, icy roads, etc. In essence, fine-tuning is the process of taking a pretrained large language model and continuing its training on a narrower, task-specific dataset. But this isn’t training from scratch—the model already knows the broad strokes of language, reasoning, coding, and world knowledge from trillions of tokens. Fine-tuning is like showing the model what *you specifically* care about: legal documents, customer support chats, scientific abstracts, or even your brand’s voice. When done right, it turns a generalist into a specialist. It lets you maintain the full general capacity of a base model while nudging it toward excellence on a narrow domain.

But why go through the effort of fine-tuning when we already have incredibly capable models? The key is **alignment and control**. Let’s say you’re a medical software provider and want your assistant to only answer questions within a strict FDA-approved knowledge base, never hallucinate, and always respond with clinical tone. Even with strong prompting, base models might still veer off. Fine-tuning allows you to embed **task norms**, **style**, **structure**, and **domain-specific behavior** into the model itself. It reduces your dependence on complex prompts (which are memory-consuming and error-prone), improves latency (by avoiding long context hacks), and gives you better **predictability** over how the model will behave. Fine-tuning is also crucial when you want your LLM to generate outputs that follow a rigid structure—say, always outputting YAML configs, or converting legal clauses into predefined templates. It can even help with **low-resource languages**, where adding focused corpora gives the model competence it wouldn’t otherwise have.

Now let’s pull the curtain back on the **mathematics and mechanics** of how fine-tuning works under the hood. When we fine-tune, we take a pretrained model and resume gradient-based optimization—typically using variants of stochastic gradient descent (like AdamW). The model is presented with input-output pairs (e.g., a question and its ideal answer), and we use a **loss function**, usually the cross-entropy loss between the predicted token distribution and the ground-truth tokens. Formally, for each token $t_i$, the model computes a probability distribution $P(t_i \mid t_1, t_2, \dots, t_{i-1})$, and we compare this with the actual next token. The loss accumulates over all tokens and is backpropagated through the entire model. Importantly, we usually fine-tune on a small learning rate and for fewer steps, so as to not forget the original general knowledge—this is known as **catastrophic forgetting**, and we try to avoid it. There are variants like **instruction tuning**, where inputs include instructions plus examples, and **reinforcement learning with human feedback (RLHF)**, which fine-tunes based not on exact tokens, but on a reward signal about whether the output is aligned with human preferences. Additionally, methods like **LoRA (Low-Rank Adaptation)** allow updating only small rank-decomposed parameter subsets (matrices), reducing the cost while achieving nearly full fine-tuning performance.

So what could go wrong if we skipped fine-tuning altogether? First, you’d be heavily reliant on prompt engineering to get task-specific behavior—which is costly, brittle, and harder to scale. Without fine-tuning, outputs might veer off into hallucinations, irrelevant digressions, or fail to follow your structural or stylistic constraints. You also lose *data efficiency*—fine-tuned models can generalize better from small, high-quality datasets than base models with massive context prompts. For safety-critical applications, skipping fine-tuning is risky: the model may behave unpredictably even with carefully crafted prompts. In highly regulated domains like healthcare, law, or finance, that’s unacceptable. On the flip side, bad fine-tuning can also degrade a model—if you overfit on a small dataset, use low-quality examples, or apply too large a learning rate, you could reduce performance on general tasks or cause mode collapse (where the model memorizes outputs and stops generalizing). That’s why fine-tuning must be approached with rigor—balancing domain specificity with retention of general reasoning power.

#### Introduction to Multimodality (Optional)

It's the extension of a language model’s purely textual understanding into other sensory domains—most commonly vision and audio—so that words can refer to pixels and sounds as naturally as they do to text. Imagine teaching a student not only to read and write, but also to recognize objects in a painting, interpret a melody, or describe a photograph. In practice, a multimodal LLM ingests images (or spectrograms, video frames, etc.) through a dedicated encoder—often a convolutional network or a Vision Transformer—that converts each patch or frame into a vector “token.” These visual (or auditory) tokens are then interleaved or concatenated with text tokens and passed jointly through the transformer’s self-attention layers. What emerges is a shared representational space where the model can, for instance, relate the phrase “golden retriever” to a specific set of pixel patterns or associate the concept of “laughter” with a particular waveform signature. This architecture allows the model to generate text about images (“captioning”), answer questions grounded in a picture (“What color is the shirt?”), or even produce simple artwork from a prompt (“Draw a red bicycle against a sunset”), all within the same sequence-modeling framework.

But why go through the complexity of bringing modalities together? Because the real world is inherently multimodal—our experiences are inseparable mixtures of sights, sounds, and words. A purely text-based model can only ever rely on descriptions; it has no direct “grounding” in visual or acoustic reality. By contrast, a multimodal model learns to align the abstract space of language with the concrete spaces of images and audio. This alignment gives it a kind of “common sense” about physical objects, spatial arrangements, textures, and acoustics, which in turn improves both its comprehension and its generation. It can disambiguate words like “bat” by referencing the shape in an image, or infer sentiment not just from the phrase “I’m fine,” but from a user’s tone of voice. Multimodality also unlocks entirely new applications—image-guided dialogue, audio captioning for accessibility, even the emergent ability to perform visual reasoning or to critique a design mockup. In many ways, it’s the missing bridge between language and perception, and crossing that bridge makes the model far more versatile, grounded, and useful.

Diving into the math, multimodal transformers typically begin by mapping each modality into the same $d$-dimensional embedding space. For images, a Vision Transformer (ViT) splits an image into $N$ patches, flattens each patch to a vector, and applies a linear projection $E_{\text{img}} \in \mathbb{R}^{(P^2 \cdot C) \times d}$—where $P$ is patch size and $C$ is channel count—to obtain $N$ image embeddings. Text tokens use their usual embedding matrix $E_{\text{text}} \in \mathbb{R}^{V \times d}$. To signal modality, each embedding has a learned **modality token type vector** added—so that the model knows whether it’s looking at words or pixels. These embeddings are concatenated into a single sequence $X = [\,x_{\text{text}}^1, \dots, x_{\text{text}}^T, x_{\text{img}}^1, \dots, x_{\text{img}}^N\,] \in \mathbb{R}^{(T+N)\times d}$, which is then processed through standard self-attention:

$
\text{Attention}(Q,K,V) = \text{softmax}\bigl(QK^\top/\sqrt{d_k}\bigr)V
$

where $Q = XW_Q,\;K = XW_K,\;V = XW_V$. Because all tokens—textual and visual—live in the same $d$-dimensional space, attention can flow freely between modalities: a text-query $q$ about “color” can attend directly to image keys and pull in the relevant pixel information from values. During training, a combination of **cross-entropy** (for text tokens) and **contrastive** or **reconstruction** losses (for image–text pairs) aligns these spaces, ensuring that semantically related text and image patches end up close together in the vector space.

If you strip multimodality out of the architecture, you lose that critical grounding: the model reverts to blind text-only reasoning, vulnerable to hallucination when describing visuals and unable to verify the content of an image. Without shared embeddings and cross-modal attention, the model can’t resolve ambiguities that depend on sight or sound, nor can it generate or critique visual content. In practice, you’d be forced into cumbersome workarounds—external vision APIs, separate audio pipelines, or manual prompt engineering—to insert “what I see is…” language into a text model. These patch-ups are brittle, introduce latency, and prevent end-to-end differentiable training, meaning the model never truly learns to connect words with sensory data. Ultimately, eliminating multimodality from the architecture turns a potential polymath back into a unidimensional word-spinner, limiting both its capabilities and the richness of its interaction with the real world.


## Step 3 - Data Analysis

It's crucial to analyze the data you're feeding to the model. Data is one of the most important things when it comes to LLMs, a bad, inconsistent data can make our model inevitable.

For this purpose, in `explanability/` folder, I've included all functions which can accelerate this process of learning.

Let's utilize them to see what this data is all about.

We'll be focusing on four key aspects of explainability: data distribution, semantic patterns, linguistic features, and token relationships.

In [ ]:
from datasets import load_dataset
# load the dataset from huggingface
ds = load_dataset("roneneldan/TinyStories")

### Pillar 1: Data Distribution

In [ ]:
from explanability import four_pillars
four_pillars.visualize(ds,pillars=[1])

#### Observation

In the first picture of visualizations, we can see that there's a bell shaped distribution cetnered around ~175 words and 152 words which is average and fine to use, majoring of texts are between 100-250 words and some outliers extend to 400+ words. The consistency in story length is well defined and the curve indicates natural variation rather than weird length cutoffs. But why? If you see that our data is of consistent text lengths, it will help our model to learn proper text structure and completion.

##### Understand Why these figures are important

1. Text length distribution is our model's reading comprehension blueprint. If 90% of content is ~175 words, our model will excel at concise texts but struggle with longer ones. But why is that? The reason behind this is because transformers process text through positional encodings that map word order to fixed dimensions. If 90% of training data is ~175 words, the model specializes in relationships within this window only, such that position embeddings beyond 200 words become under-trained, kind of like overfitting on concise words only. The self attention mechanism also devevlops length specific sparsity patterns which means, short texts are learned through dense global attention while long texts are learned through localized attention windows (neightboring words). The Memory allocation determines default "attention span" needs, while outliers dictate emergency capacity. Learning rhythm creates steady training batches for smooth gradient flow; the tight cluster (100-250 words) ensures this, while the long tail prevents complacency. If the average text length is around 100-250 words, 85% of attention heads are optimized while if more, 15% heads handle outliers, like a city building 85% of its hospitals in downtown while neglecting suburbs.

2. Character length distribution reveals the atomic structure of our text. Short texts under 500 characters optimize tokenization efficiency, while texts over 1000 characters inform batch sizing decisions. Why? Because character counts directly impact tokenization strategies - languages with complex characters (like Chinese) yield fewer tokens per character than space-delimited languages (like English). The distribution shape acts as a processing compass: narrow bell curves enable fixed-length optimization, while bimodal distributions suggest mixed content types requiring adaptive processing. GPU memory allocation becomes predictable with consistent character lengths, while widely varying lengths may require dynamic batching to prevent OOM errors. Character/word ratio variations signal language diversity or specialist terminology that influences vocabulary design. This isn't just counting - it's our model's processing budget blueprint.

3. Word frequency charts inventory our dataset's vocabulary landscape. The power-law distribution is fundamental - typically 20% of unique words account for 80% of occurrences. This matters because high-frequency words shape our model's attention patterns most strongly during training. Domain expertise emerges through specialized terminology frequencies. Medical texts show high frequencies of anatomical terms, legal texts prioritize procedural vocabulary - these signature words define what field our model will excel in. The cutoff threshold between common and rare words determines vocabulary size, directly impacting model parameter count. Uncommon words (appearing <5 times) often represent the specialized knowledge edge cases that separate exceptional models from average ones. The long tail isn't just noise - it contains the domain-specific precision terms that enable expert-level responses, while the head contains structural words that define grammatical patterns. Balancing both determines our model's communication style.

4. The word cloud transforms frequency data into intuitive visual territory. Size-weighted words reveal conceptual landmarks, but why does this matter? Because human stakeholders process visual patterns faster than numerical tables - the cloud becomes a shared communication bridge between technical and non-technical team members. Visual clustering reveals semantic neighborhoods that might be missed in tabular analysis. Words that appear disproportionately large signal potential concept skew that may bias our model's outputs. The absence of expected domain terms is equally important - these represent blind spots in our training data. Color and positioning aren't just aesthetic choices - they help identify concept clusters and thematic boundaries. Unlike raw statistics, word clouds trigger intuitive pattern recognition, helping identify unexpected relationships between concepts. This visualization serves as both a technical diagnostic tool and an interpretability bridge for explaining our model's knowledge territory to non-technical stakeholders.




### Pillar 2: Semantic Patterns

In [ ]:
four_pillars.visualize(ds,pillars=[2])

#### Observation

In this Pillar 2 visualization, we can see five distinct topics extracted from the TinyStories dataset. Each topic is characterized by different character names and associated concepts - Topic 1 centers around "lucy" with nature elements like "frog" and "bug," while Topic 2 focuses on family interactions with "lily," "said," "mom," and "timmy." The document-topic distribution shows most stories blend multiple topics rather than focusing on just one, which teaches the model how to transition between themes naturally. This semantic mapping ensures the model learns contextual relationships between words and concepts rather than treating language as isolated tokens, enabling more coherent and contextually appropriate story generation.

##### Understand Why these figures are important

1. Topic keyword distributions reveal the conceptual DNA of our training corpus. The image shows 5 distinct topics with character names (lucy, lily, benny, tom, jack) dominating each cluster, signaling character-centric narratives. But why does this matter? Topic modeling helps our LLM understand conceptual relationships between words - "frog" and "bug" appearing in Topic 1 teaches our model these belong to a similar context, creating semantic neighborhoods in embedding space. When our model later encounters "frog," it knows to activate neurons related to similar small creatures. These distributions guide attention patterns during both training and inference: our model learns to predict that after "lucy" and "bug" appear, words like "frog" become more likely. This semantic blueprint prevents hallucinations by establishing valid conceptual connections.

2. Topic coherence determines how clearly defined our semantic categories are. Notice how Topic 2 includes "lily," "mom," "said," and "timmy" - suggesting family dialogue scenarios. Strong topic coherence means our model learns tightly bound concept clusters rather than loose associations. Why? Because transformer attention mechanisms operate most effectively when related concepts have clear distinctions and boundaries. Fuzzy topics lead to fuzzy attention, creating generation uncertainty. During fine-tuning, concept-coherent data helps our model develop specialized attention heads that activate for specific semantic domains. Models trained on data with poor topic coherence typically produce outputs that drift between subjects unpredictably.

3. The document-topic distribution (colorful vertical bars) maps how topics blend in each document. Most stories contain multiple topics, with orange (Topic 2) appearing most frequently. This pattern matters profoundly because it teaches our model natural topic transitions rather than artificially segmented content. Real stories don't contain just one topic - they weave multiple themes together. The blending proportions (some documents have 70% Topic 2, 30% Topic 4) train our model's internal state transitions, enabling coherent paragraph-to-paragraph flow. Models trained on single-topic documents often generate content that feels unnaturally focused or abruptly switches subjects. This distribution pattern is our model's roadmap for narrative pacing and thematic blending.

4. Topic balance across the corpus shapes our model's conceptual priorities. Topic 2 (centered around "lily") dominates many documents while Topic 5 appears less frequently. This imbalance creates an implicit prior in our model - it will naturally gravitate toward generating content similar to dominant topics. Why does this matter? Because unintentional biases in topic distribution become amplified during generation, potentially creating repetitive outputs. The semantic landscape should match our intended generation domain - if creating a diverse storytelling model, you want relatively balanced topics. Recognizing imbalance lets you either rebalance our dataset or implement techniques like topic-weighted loss functions to compensate during training.

5. Keyword frequency within topics signals semantic centrality. "Lucy" dominates Topic 1, while "bug" and "frog" appear less frequently. This teaches our model hierarchical concept relationships - primary entities (characters) and their associated attributes or actions. Models learn to structure attention around these hierarchies, making main concepts focal points for generation. This matters because well-structured semantic hierarchies produce more coherent, human-like text with appropriate subject-object relationships. The strongest keyword associations become default patterns our model falls back on when generation uncertainty increases. Understanding these patterns helps explain and predict our model's generation tendencies and failure modes.



### Pillar 3: Linguistic Features

In [ ]:
four_pillars.visualize(ds,pillars=[3])

#### Observation
The linguistic features in Pillar 3 reveal a dataset optimized for children's content. The bimodal Flesch score distribution (peaks at 85-90 and 95-100) shows the text maintains very high readability while incorporating some variety in complexity. The US Grade Level concentrating around 4-5 perfectly targets elementary school readers. Sentence structure is consistent and approachable (average 10.87 words), avoiding overly complex constructions that might confuse young readers. The predominantly short word lengths (3.87 characters average) ensure vocabulary accessibility. Together, these features create an ideal linguistic profile for a model designed to generate children's stories that are easy to understand yet still engaging and varied in structure.

##### Understand Why these figures are important
1. Flesch Reading Ease scores reveal our dataset's cognitive accessibility profile. The bimodal distribution with peaks at 85-90 and 95-100 indicates two distinct complexity levels within very easy reading material. But why does this matter? Transformer models develop different reasoning pathways when exposed to varied complexity levels. The 95-100 peak texts teach basic language patterns (subject-verb-object, common words, direct statements), while the 85-90 peak introduces slightly more complex structures (compound sentences, broader vocabulary). This linguistic stepping stone approach is crucial because it prevents our model from becoming one-dimensional in its generation style. Without this dual-peak distribution, our model might produce overly simplistic or inconsistently complex outputs. These scores directly influence how our model learns to balance clarity with sophistication - too many complex texts would teach verbose outputs, while too many simple texts would limit expression range.

2. US Grade Level distribution maps the educational accessibility of our corpus. The concentration around grades 4-5 (with average 4.25) aligns perfectly with elementary school reading levels. This matters because it determines our model's implicit audience awareness. Models learn implicit tone-audience mapping - grade 2-3 content teaches simple explanations suitable for beginners, while grade 5-6 content demonstrates how to articulate more nuanced concepts. Why is this important? Because models trained exclusively on academic papers (grade 15+) struggle to simplify concepts for general audiences, while those trained on only grade 1-2 texts lack vocabulary for complex topics. The relatively narrow grade range (mostly 3-6) suggests our model will develop consistent educational voice rather than erratic complexity shifts. This educational calibration becomes our model's default communication level during generation.

3. Sentence Length distribution reveals our data's syntactic rhythm blueprint. The clearly defined peak around 10-11 words per sentence (average 10.87) shows consistent, approachable sentence structures. This pattern matters deeply because transformers learn optimal attention span patterns from sentence boundaries. Short, consistent sentences teach models to create digestible information chunks rather than overwhelming paragraph-long statements. Each sentence length range teaches different skills: 5-7 word sentences build concise expression, 10-12 word sentences balance detail with clarity, and occasional 15+ word sentences demonstrate complex idea linking. Models trained on wildly varying sentence lengths often generate unpredictable rhythm shifts that feel unnatural. This distribution shapes our model's natural "breath points" during generation - the pauses and breaks that make generated text feel human rather than mechanical.

4. Word Length distribution exposes our vocabulary's complexity fingerprint. The spikes at 2-4 characters with average 3.87 characters indicate predominantly simple, accessible vocabulary. But why does this shape our model? Word length directly correlates with language accessibility and concept abstraction. The prevalence of 3-letter words trains our model to prioritize common, concrete terms over specialized vocabulary. Models learn implicit word-complexity associations - shorter words for foundational concepts, longer words for specialized topics. This distribution ensures our model develops a vocabulary appropriate for its target audience (children, based on the grade levels). Models trained on datasets with much higher average word lengths (6+ characters) tend to generate unnecessarily complex or jargon-heavy content, while those trained on only very short words lack precision. our word length profile trains balance between accessibility and precision.

### Pillar 4: Token Relationships


In [ ]:
four_pillars.visualize(ds,pillars=[4])

#### Observation

The visualizations in Pillar 4 show that our dataset is rich in familiar storybook phrases and strong word associations, such as "once upon a time," "there was a," and predictable continuations after common words like "the." The co-occurrence matrix confirms that key characters and actions are tightly linked, reinforcing narrative structure. These patterns ensure our model learns to generate text that is fluent, contextually relevant, and stylistically consistent with children’s stories, rather than producing random or disjointed sequences.

##### Understand Why these figures are important

1. Bigram and Trigram Frequency charts (top left and right) reveal the most common two- and three-word sequences in our dataset. Seeing phrases like "was a," "there was a," and "once upon a time" dominate shows that our training data is rich in narrative patterns and storybook language. Why does this matter? These frequent n-grams act as linguistic building blocks-our model learns to generate coherent, natural-sounding sentences by internalizing these common phrase templates. Repeated exposure to such patterns helps the model reliably predict what comes next in a story, improving fluency and narrative flow. If our dataset lacked these recurring structures, the model might generate disjointed or awkward sentences.

2. Word Co-occurrence Matrix (bottom left) visualizes how often pairs of words appear together across the corpus. Bright spots indicate strong associations, such as between pronouns ("he," "she") and verbs ("said," "was"), or between characters and their actions. This matters because transformers learn context by modeling how tokens relate to their neighbors. Strong co-occurrence signals help the model understand which words naturally cluster together, reinforcing contextual awareness and thematic consistency. Sparse or random co-occurrence patterns would make it harder for the model to learn meaningful relationships, potentially leading to incoherent outputs.

3. Top Words Following "the" (bottom right) highlights the most likely continuations after a common article. Words like "park," "little," and "bird" frequently follow "the," reflecting the story’s subject matter and vocabulary focus. This is important because it shows the model is exposed to realistic, domain-relevant continuations, which will help it generate plausible and contextually appropriate text. If the words following "the" were highly unpredictable or unrelated, it would signal a lack of thematic cohesion in our dataset.

4. N-gram and Co-occurrence Patterns as Model Guidance: These token relationships serve as the scaffolding for our model’s language generation. Frequent bigrams and trigrams become the backbone of sentence construction, while strong co-occurrence patterns teach the model how to maintain context over longer passages. This is especially crucial for story generation, where maintaining narrative consistency and logical progression is key.



### Summary
The four pillars collectively form a linguistic DNA profile for our LLM training data:

1. Distribution Pillar ensures our model’s "attention span" matches real-world use cases through optimized text lengths.  
2. Semantic Pillar builds conceptual coherence by mapping topic relationships and thematic blending.  
3. Linguistic Pillar calibrates readability and syntactic patterns for audience-appropriate generation.  
4. Token Pillar engineers fluent output through learned phrase templates and word associations.  

Our dataset demonstrates strong foundational suitability for training a children’s story LLM. The text length distribution (100-250 words) aligns with young readers’ attention spans, semantic patterns teach natural topic transitions, linguistic features enforce age-appropriate complexity, and token relationships establish essential storytelling phrases.  

## Step 4 - Training the Model

It's time for training the actual model, let's build it!

### The Model Structure

Let's finally join all the parts of the model together

In [ ]:
# Implements a single transformer layer with self-attention and feedforward sublayers
class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, activation='gelu'):
        super().__init__()
        # Add dropout parameter to constructor
        self.dropout = nn.Dropout(dropout)
        # Pre-norm residual connection around multi-head attention
        self.attention = PreNormResidual(d_model,
            MultiHeadSelfAttention(d_model, num_heads))
        # Pre-norm residual connection around feedforward network
        self.ffn = PreNormResidual(d_model,
            FeedForward(d_model, d_ff, activation))

    def forward(self, x, mask=None, cache=None, layer_idx=0, seq_pos=0):
        # Apply dropout after attention
        x = self.attention(x, mask, cache, layer_idx, seq_pos)
        x = self.dropout(x)
        # Apply dropout after FFN
        x = self.ffn(x)
        x = self.dropout(x)
        return x

# Main transformer model combining embeddings, positional encoding, and N layers
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=768, num_layers=12,
                 num_heads=12, d_ff=3072, dropout=0.1, max_seq_len=512):
        super().__init__()
        # Add dropout parameter and pass to layers
        self.dropout = nn.Dropout(dropout)
        # Convert token IDs to continuous embeddings
        self.embeddings = InputEmbedding(vocab_size, d_model)
        # Add positional information to embeddings
        self.positional = PositionalEncoding(d_model, max_seq_len)

        # Stack multiple transformer layers with dropout
        self.layers = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])

        # Final projection to vocabulary size with weight tying
        self.output = OutputProjection(d_model, vocab_size,
                                     self.embeddings.embedding.weight)

        # Initialize weights using Kaiming normal for linear layers
        self.apply(self._init_weights)

    def forward(self, x, mask=None, cache=None, seq_pos=0):
        # Embed tokens and add positional information
        x = self.embeddings(x)
        x = self.positional(x)
        x = self.dropout(x)  # Apply dropout after embeddings

        # Process through each transformer layer sequentially
        for layer_idx, layer in enumerate(self.layers):
            x = layer(x, mask, cache, layer_idx, seq_pos)

        # Convert final hidden states to vocabulary logits
        return self.output(x)

    # Weight initialization strategy from GPT-2 paper
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode='fan_in', nonlinearity='linear')
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0, std=0.02)



The `TransformerBlock` class implements a single layer of the modern transformer architecture, incorporating the critical components that have made transformers the foundation of state-of-the-art language models. When initialized, the class constructs a sophisticated processing unit that combines self-attention mechanisms with feedforward neural networks, each wrapped in pre-normalization residual connections to facilitate training of deep networks. The constructor takes several key parameters: `d_model` specifies the dimensionality of the model's hidden representations (typically ranging from 512 to several thousand in large models), `num_heads` determines how many parallel attention mechanisms operate simultaneously (allowing the model to focus on different aspects of the input), `d_ff` sets the intermediate dimensionality of the feedforward network (usually 4x larger than `d_model` to provide sufficient expressivity), and `dropout` controls regularization strength to prevent overfitting. The architecture follows the "pre-norm" pattern, where layer normalization is applied before each sublayer rather than after (as in the original transformer paper), which has been shown to improve training stability in deep models. The forward pass implements a sequential flow where input tensors first pass through the self-attention sublayer—allowing each position to gather information from all other positions—followed by dropout regularization to prevent co-adaptation of features. The output then proceeds through the position-wise feedforward network, which applies the same transformation independently to each position, providing additional representational power through its non-linear activations. After another application of dropout, the processed tensor is returned. Importantly, both the attention and feedforward components leverage residual connections (implemented within the `PreNormResidual` wrapper) that preserve gradient flow through deep networks by providing direct pathways for backpropagation. The forward method also accepts optional parameters like `mask` (to control which positions can attend to others, critical for causal/autoregressive generation), `cache` (to store key-value pairs for efficient sequential generation), and positional indicators (`layer_idx` and `seq_pos`) that enable optimizations like KV caching during text generation. This design encapsulates the core innovation of transformers: replacing recurrent or convolutional architectures with pure attention mechanisms supplemented by position-aware feedforward networks, creating a highly parallelizable yet expressive building block that scales effectively to massive model sizes.

The `TransformerModel` class implements a comprehensive neural network architecture that represents the core foundation of modern language models. At initialization, the class establishes the complete computational backbone that has revolutionized natural language processing—from token embeddings to final vocabulary projections. The constructor accepts several crucial hyperparameters that define the model's capacity and behavior: `vocab_size` determines the input/output dimensionality based on the tokenizer's vocabulary; `d_model` (typically 768 in medium-sized models) establishes the continuous vector dimension that tokens will occupy throughout the network; `num_layers` specifies the stack depth (often 12 in base models but scaling to hundreds in larger variants); `num_heads` (commonly 12) enables parallel attention computation across different representation subspaces; `d_ff` (usually 4× the model dimension) provides the intermediate dimensionality for position-wise feed-forward networks; `dropout` (commonly 0.1) controls regularization; and `max_seq_len` (default 512) defines the maximum sequence length the positional encoding can handle. After initialization, the model assembles a carefully structured pipeline: first, an `InputEmbedding` layer transforms discrete token IDs into learned, continuous vector representations; then, a `PositionalEncoding` layer injects crucial sequence order information into these otherwise position-agnostic embeddings. The transformer's heart lies in its layered structure, implemented as a `nn.ModuleList` containing multiple `TransformerBlock` instances, each combining self-attention and feed-forward sublayers wrapped in pre-normalization residual connections. A critical optimization appears in the output projection layer, which ties its weight matrix to the embedding layer's weights (`self.embeddings.embedding.weight`), effectively halving the parameter count for these layers while enforcing consistency in how tokens are represented at input and output. The model concludes with a careful weight initialization strategy derived from the GPT-2 paper—linear layers receive Kaiming normal initialization (adjusted for linear activation functions) while embedding layers get a small Gaussian initialization (mean 0, std 0.02). During the forward pass, the model processes input token IDs through the embedding layer, adds positional information, applies dropout for regularization, then sequentially passes the representation through each transformer block, tracking the layer index for potential key-value caching during generation. Throughout this process, attention masks can be applied (e.g., causal masks for autoregressive generation), and an optional cache parameter enables efficient sequential generation by storing intermediate key-value pairs. The final representation is projected back to vocabulary logits through the output layer, completing the transformation from input tokens to predicted next-token probabilities. This architecture embodies the core innovation of transformers: replacing recurrent connections with parallel self-attention mechanisms that can model long-range dependencies while maintaining computational efficiency.

### Training with the dataset

In [ ]:
# Handles training loop with mixed precision and gradient clipping
class LLMTrainer:
    def __init__(self, model, lr=3e-4, grad_clip=1.0, device='cuda', grad_accum_steps=4):
        self.device = device
        print(f"Using device: {device}")
        self.true_losses = []
        self.model = model.to(self.device)
        self.optim = torch.optim.AdamW(model.parameters(), lr=lr,
                                  weight_decay=0.1, betas=(0.9, 0.98))
        self.grad_clip = grad_clip
        self.grad_accum_steps = grad_accum_steps
        self.current_step = 0
        if device == 'cuda':
            self.scaler = torch.cuda.amp.GradScaler()
        else:
            self.scaler = None

    def train_step(self, batch):
        inputs, targets = batch
        inputs = inputs.to(self.device)
        targets = targets.to(self.device)

        # Use autocast only for CUDA
        if self.device == 'cuda':
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                logits = self.model(inputs)
                # Calculate true loss before scaling
                true_loss = compute_loss(logits, targets)
                # Scale loss for gradient accumulation
                loss = true_loss / self.grad_accum_steps

            # Use scaler for CUDA
            self.scaler.scale(loss).backward()

            # Only update weights after accumulating gradients
            self.current_step += 1
            if self.current_step % self.grad_accum_steps == 0:
                self.scaler.unscale_(self.optim)
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
                self.scaler.step(self.optim)
                self.scaler.update()
                self.optim.zero_grad(set_to_none=True)
        else:
            # Standard CPU training
            logits = self.model(inputs)
            loss = compute_loss(logits, targets)
            loss = loss / self.grad_accum_steps
            loss.backward()

            self.current_step += 1
            if self.current_step % self.grad_accum_steps == 0:
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
                self.optim.step()
                self.optim.zero_grad(set_to_none=True)

        self.true_losses.append(true_loss.item())
        # Return average of recent true losses
        return sum(self.true_losses[-10:]) / min(len(self.true_losses), 10)

        # return loss.item() * self.grad_accum_steps  # Return unscaled loss for logging

# Autoregressive text generation with temperature and top-k sampling
def generate(model, prompt, tokenizer, max_length=50,
            temperature=0.7, top_k=40, device='cuda'):
    model.eval()
    tokens = tokenizer.encode(prompt)
    # Initialize KV cache for efficient sequential generation
    cache = KVCache(num_layers=len(model.layers),
                  max_seq_len=max_length,
                  num_heads=model.layers[0].attention.num_heads,
                  head_dim=model.layers[0].attention.d_head)

    with torch.no_grad():
        for _ in range(max_length):
            # Process only the last token in the sequence
            inputs = torch.tensor([tokens[-1:]]).to(device)
            # Forward pass using cached keys/values
            logits = model(inputs, cache=cache, seq_pos=len(tokens)-1)
            logits = logits[:, -1, :] / temperature

            # Restrict sampling to top-k most probable tokens
            v, ix = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float('Inf')

            # Sample from filtered distribution
            probs = torch.nn.functional.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            tokens.append(next_token.item())

            # Stop if end-of-sequence token is generated
            if next_token.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(tokens)


The `LLMTrainer` class implements a sophisticated training framework for large language models that incorporates several modern optimization techniques to balance computational efficiency with training stability. During initialization, the class configures the training environment by moving the model to the specified device (CUDA GPU or CPU), initializing an AdamW optimizer with carefully chosen hyperparameters (learning rate of 3e-4, weight decay of 0.1, and beta values of 0.9 and 0.98 for momentum and RMS scaling), and setting up gradient accumulation (defaulting to 4 steps) to simulate larger batch sizes without requiring proportionally more memory. When running on CUDA devices, the class also creates a `GradScaler` object from PyTorch's automatic mixed precision (AMP) package, which enables training in memory-efficient 16-bit floating point precision while maintaining numerical stability. The core functionality resides in the `train_step` method, which processes a single batch of inputs and targets, adaptively selecting between CUDA-optimized or standard CPU training paths. In the CUDA path, computation operates within an `autocast` context that automatically converts eligible operations to float16, significantly reducing memory usage and often accelerating computation on modern GPUs. After computing the model's logits and calculating the true loss using a cross-entropy function, the method scales this loss by the gradient accumulation factor (effectively implementing micro-batching) before performing backpropagation through the `scale` method of the `GradScaler`. The trainer then increments an internal step counter and, when sufficient gradients have accumulated, unscales the gradients, applies gradient clipping to prevent exploding gradients (constrained to `grad_clip`, typically 1.0), and updates the model parameters using the optimizer before zeroing the gradients. For CPU training, the method follows a similar flow but without mixed precision components. In both paths, the true (unscaled) loss is recorded in a running history, and the method returns a smoothed average of the most recent losses to provide a more stable training signal. Complementing this trainer is the `generate` function, which implements efficient autoregressive text generation with various decoding strategies. It first puts the model in evaluation mode to disable dropout, then encodes the input prompt into tokens and initializes a key-value cache to store intermediate attention computations. Within a no-gradient context for efficiency, the function iteratively generates new tokens by: feeding only the most recently generated token into the model (rather than recomputing the entire sequence), scaling the resulting logits by a temperature parameter to control randomness, restricting the sampling space to only the top-k most likely tokens for quality control, sampling the next token according to the resulting probability distribution using multinomial sampling, and appending this token to the sequence until either the maximum length is reached or an end-of-sequence token is generated. This combination of sophisticated training mechanisms and generation capabilities enables efficient development of high-quality language models while addressing common challenges like memory constraints, numerical instability, and generation quality.

### Model Checkpointing & Resumption

In [ ]:
def save_checkpoint(model, trainer, path, epoch, step, config, tokenizer=None):
    checkpoint = {
        'model_state': model.state_dict(),
        'optim_state': trainer.optim.state_dict(),
        'scaler_state': trainer.scaler.state_dict() if trainer.scaler else None,
        'epoch': epoch,
        'step': step,
        'config': config,
        # Store tokenizer configuration in a compatible way
        'tokenizer_config': {
            'vocab_size': tokenizer.vocab_size,
            'added_tokens': list(tokenizer.added_tokens_encoder.keys()) if hasattr(tokenizer, 'added_tokens_encoder') else [],
            'special_tokens': tokenizer.special_tokens_map
        } if tokenizer else None
    }
    torch.save(checkpoint, path)
    print(f"Saved checkpoint to {path}")

def load_checkpoint(path, model, trainer, tokenizer=None, device='cuda'):
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state'])
    trainer.optim.load_state_dict(checkpoint['optim_state'])

    if trainer.scaler and 'scaler_state' in checkpoint and checkpoint['scaler_state']:
        trainer.scaler.load_state_dict(checkpoint['scaler_state'])

    # No need to modify tokenizer - we're using a pre-trained one

    print(f"Loaded checkpoint from {path} (epoch {checkpoint['epoch']})")
    return checkpoint['epoch'], checkpoint['step']

The `save_checkpoint` and `load_checkpoint` functions implement a comprehensive checkpoint management system for transformer-based language models, ensuring training continuity and model reproducibility across sessions. The `save_checkpoint` function encapsulates the complete training state by creating a dictionary that preserves multiple critical components: the model's learned parameters through `model.state_dict()`, the optimizer's internal state via `trainer.optim.state_dict()` (which includes momentum buffers and learning rate adaptations essential for optimization trajectory), and the gradient scaler's state through `trainer.scaler.state_dict()` when using mixed precision training. Beyond these core training states, it records contextual metadata including the current epoch number, step counter within that epoch, and model architecture configuration parameters. Particularly noteworthy is its careful handling of tokenizer information, where it extracts essential vocabulary statistics (total size, custom tokens, and special token mappings) while avoiding serializing the entire tokenizer object, which might contain implementation-specific details that could change between framework versions. After constructing this comprehensive state snapshot, the function serializes it to the specified file path using PyTorch's serialization utility, creating a persistent record that can survive across training sessions, computing environment changes, or even infrastructure migrations. The complementary `load_checkpoint` function reverses this process, beginning by deserializing the saved checkpoint with explicit device mapping to accommodate training resumption on different hardware. It methodically restores each component of the training state: loading model parameters, reinstating optimizer state (preserving momentum trajectories and adaptive learning rates exactly where they left off), and when applicable, restoring the mixed precision scaler's state to maintain numerical stability parameters. The function deliberately avoids modifying the tokenizer object, operating under the assumption that a consistent pre-trained tokenizer is being used across sessions, which prevents potential vocabulary misalignments. This checkpoint system balances completeness with portability, ensuring that training can be paused and resumed with minimal loss of progress while maintaining the exact optimization trajectory, which is particularly important for large language models where training may span days or weeks and might need to be interrupted for resource scheduling or hardware maintenance. The returned epoch and step values allow the training loop to precisely continue from where it left off, supporting workflows that might involve dynamic resource allocation or distributed training environments.

### Training the model

In [ ]:
def evaluate_diversity(model, tokenizer, num_samples=5, device='cuda'):
    """Evaluate text diversity by generating multiple samples and analyzing them"""
    prompts = [
        "Once upon a time",
        "The little dog",
        "She looked at the",
        "Tom was sad because",
        "In the forest"
    ]

    all_generations = []
    unique_ngrams = set()
    total_ngrams = 0

    model.eval()
    with torch.no_grad():
        for prompt in prompts:
            # Generate multiple completions for each prompt
            for _ in range(num_samples):
                generated = generate(model, prompt, tokenizer,
                                    max_length=50,
                                    temperature=0.8,  # Slightly higher temperature for diversity
                                    device=device)

                # Store full generation
                all_generations.append(generated)

                # Extract n-grams for diversity analysis
                tokens = generated.split()
                for i in range(len(tokens) - 2):  # Use trigrams for diversity measure
                    trigram = " ".join(tokens[i:i+3])
                    unique_ngrams.add(trigram)
                    total_ngrams += 1

    # Calculate diversity metrics
    if total_ngrams > 0:
        diversity_score = len(unique_ngrams) / total_ngrams
    else:
        diversity_score = 0

    repetition_rate = 1.0 - diversity_score

    print(f"\n===== Generation Diversity Analysis =====")
    print(f"Total samples generated: {len(all_generations)}")
    print(f"Unique trigrams: {len(unique_ngrams)}")
    print(f"Repetition rate: {repetition_rate:.4f} (lower is better)")

    print("\nSample generations:")
    for i, text in enumerate(all_generations[:3]):  # Show first 3
        print(f"\nSample {i+1}: {text}\n{'-'*40}")

    return {
        'diversity_score': diversity_score,
        'repetition_rate': repetition_rate,
        'generations': all_generations
    }

In [ ]:

# Create a directory for checkpoints
os.makedirs("models", exist_ok=True)

# Find the latest checkpoint if it exists
def find_latest_checkpoint(checkpoint_dir="models"):
    checkpoints = glob.glob(f"{checkpoint_dir}/model_epoch*_batch*.pt")
    if not checkpoints:
        return None

    # Sort by modification time (most recent first)
    latest = max(checkpoints, key=os.path.getmtime)
    return latest

def train_model(model, trainer, train_loader, tokenizer, num_epochs=3,
                checkpoint_dir="models", checkpoint_freq=500):
    # Try to find latest checkpoint
    latest_checkpoint = find_latest_checkpoint(checkpoint_dir)
    start_epoch = 0
    start_batch = 0

    # Resume from checkpoint if available
    if latest_checkpoint:
        start_epoch, start_batch = load_checkpoint(
            latest_checkpoint, model, trainer, tokenizer,
            device='cuda' if torch.cuda.is_available() else 'cpu'
        )
        print(f"Resuming from epoch {start_epoch+1}, batch {start_batch}")
    else:
        print("Starting training from scratch")

    total_batches = len(train_loader)
    print(f"Training on {total_batches} batches per epoch")

    # Track diversity scores across epochs
    epoch_diversity_scores = []

    for epoch in range(start_epoch, num_epochs):
        total_loss = 0
        batch_count = 0

        # Skip to the right batch if resuming mid-epoch
        skip_batches = start_batch if epoch == start_epoch else 0

        for batch_idx, batch in enumerate(train_loader):
            # Skip batches we've already processed
            if batch_idx < skip_batches:
                continue

            loss = trainer.train_step(batch)
            total_loss += loss
            batch_count += 1

            if batch_idx % 100 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx}/{total_batches} | Loss {loss:.4f}")

            # Save checkpoint at regular intervals
            if batch_idx > 0 and batch_idx % checkpoint_freq == 0:
                checkpoint_path = f"{checkpoint_dir}/model_epoch{epoch+1}_batch{batch_idx}.pt"
                config = {
                    "vocab_size": model.embeddings.embedding.weight.size(0),
                    "d_model": model.embeddings.d_model,
                    "num_layers": len(model.layers),
                    "num_heads": model.layers[0].attention.sublayer.num_heads,
                    "d_ff": model.layers[0].ffn.sublayer.linear1.out_features
                }
                save_checkpoint(model, trainer, checkpoint_path, epoch, batch_idx, config, tokenizer)

        # Calculate average loss for epoch
        avg_loss = total_loss / batch_count if batch_count > 0 else float('inf')
        print(f"Epoch {epoch+1} completed | Average loss: {avg_loss:.4f}")

        # Save checkpoint at end of each epoch
        checkpoint_path = f"{checkpoint_dir}/model_epoch{epoch+1}_final.pt"
        config = {
            "vocab_size": model.embeddings.embedding.weight.size(0),
            "d_model": model.embeddings.d_model,
            "num_layers": len(model.layers),
            "num_heads": model.layers[0].attention.sublayer.num_heads,
            "d_ff": model.layers[0].ffn.sublayer.linear1.out_features
        }
        save_checkpoint(model, trainer, checkpoint_path, epoch, total_batches, config, tokenizer)

        # Reset start_batch after completing the resumed epoch
        start_batch = 0

        # Generate sample text after each epoch
        prompt = "Once upon a time"
        with torch.no_grad():
            generated = generate(model, prompt, tokenizer, max_length=50)
        print(f"Sample generation:\n{generated}\n")

        # Perform diversity evaluation
        diversity_metrics = evaluate_diversity(
            model, tokenizer,
            num_samples=3,
            device=trainer.device
        )
        epoch_diversity_scores.append(diversity_metrics['diversity_score'])

        # Early sign of overfitting: diversity decreasing substantially
        if len(epoch_diversity_scores) >= 2 and epoch_diversity_scores[-1] < epoch_diversity_scores[-2] * 0.7:
            print("WARNING: Significant drop in text diversity detected - possible overfitting!")

    print("\nDiversity score progression across epochs:")
    for i, score in enumerate(epoch_diversity_scores):
        print(f"Epoch {i+1}: {score:.4f}")

    return model
# Use the new training function
model = TransformerModel(
    vocab_size=tokenizer.vocab_size + len(tokenizer.added_tokens_encoder),
    d_model=256, # Reduced model size for faster training
    num_layers=4,   # Reduced number of layers for faster training
    num_heads=4,    # Reduced number of heads for faster training
    d_ff=1024, # Reduced feedforward size for faster training
    dropout=0.2  # Increased dropout for better generalization
)

# Initialize the trainer with the model and training parameters
trainer = LLMTrainer(model, lr=3e-5, grad_clip=1.0, device='cuda' if torch.cuda.is_available() else 'cpu')

# Start or resume training
train_loader = DataLoader(tok_ds, batch_size=16, shuffle=True, collate_fn=collate_fn, drop_last=False)

print(f"Total training batches: {len(train_loader)}")

# Train the model for a specified number of epochs
trained_model = train_model(model, trainer, train_loader, tokenizer, num_epochs=3)

The provided code implements a comprehensive training and evaluation system for language models, with particular focus on monitoring and maintaining text generation diversity. The `evaluate_diversity` function serves as a crucial analytical tool that quantitatively measures a model's capacity to generate varied and non-repetitive text across different prompt contexts. It operates by systematically generating multiple text completions for each of five story-starter prompts using a slightly elevated temperature setting (0.8) to encourage creativity. For each generated text, the function extracts all possible trigrams (three consecutive word sequences) and tracks both their uniqueness and total count across the entire sample pool. This extraction process enables the calculation of a diversity score—defined as the ratio of unique trigrams to total trigrams—which effectively quantifies lexical variety in the model's outputs; a higher score indicates greater diversity, while a lower score suggests repetitive phrasing. The `train_model` function implements a robust training loop with intelligent checkpointing mechanisms that ensure training continuity even in the event of interruptions. It first attempts to locate and load the most recent checkpoint using timestamp-based sorting, resuming from the exact batch and epoch where training previously halted. For each epoch, the function methodically processes training batches, tracking losses and periodically saving checkpoints at configurable intervals (default: every 500 batches). Critically, the function integrates diversity evaluation into the training process, calling `evaluate_diversity` after each epoch to monitor the model's generation quality over time. This integration enables early detection of overfitting through a novel heuristic: if diversity scores drop by more than 30% between consecutive epochs, the function issues a warning about potential overfitting. The function maintains comprehensive training state including model parameters, optimizer state, gradient scaler state (for mixed precision training), tokenizer configuration, and architectural hyperparameters in each checkpoint, ensuring complete reproducibility. The model instantiation and training initialization code demonstrates practical hyperparameter choices for resource-constrained environments, creating a relatively small transformer (256-dimensional embeddings, 4 layers, 4 attention heads) with increased dropout (0.2) to combat overfitting. Together, these components form an end-to-end system that not only trains language models but also continuously monitors their generative diversity—a critical quality metric that traditional loss functions alone cannot capture—enabling proactive intervention when models begin to produce increasingly formulaic or repetitive text.

### Evaluation

In [ ]:
# Validation metrics for language model evaluation
class ValidationMetrics:
    def __init__(self, device='cuda'):
        self.losses = []
        self.top1_acc = []  # Token-level accuracy
        self.top5_acc = []  # Top-5 accuracy
        self.device = device

    def update(self, logits, targets):
        with torch.no_grad():
            loss = compute_loss(logits, targets).item()
            self.losses.append(loss)

            # Calculate prediction rankings
            probs = torch.softmax(logits, dim=-1)
            _, preds = torch.topk(probs, k=5, dim=-1)
            targets = targets.unsqueeze(-1)

            # Compute accuracy metrics
            top1 = (preds[..., :1] == targets).any(-1)
            top5 = (preds == targets).any(-1)

            self.top1_acc.extend(top1.cpu().tolist())
            self.top5_acc.extend(top5.cpu().tolist())

    def compute(self):
        return {
            'perplexity': torch.exp(torch.tensor(self.losses).mean()).item(),
            'loss': np.mean(self.losses),
            'top1_acc': np.mean(self.top1_acc),
            'top5_acc': np.mean(self.top5_acc)
        }

# Validation loop for model evaluation
def validate(model, val_loader, device='cuda'):
    model.eval()
    metrics = ValidationMetrics(device)

    with torch.no_grad():
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)

            logits = model(inputs)
            metrics.update(logits, targets)

    model.train()
    return metrics.compute()

class LLMEvaluator:
    def __init__(self, model, tokenizer, device='cuda'):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.model.to(device)
        self.model.eval()

    def compute_metrics(self, data_loader):
        """Compute standard validation metrics on a dataset"""
        metrics = ValidationMetrics(self.device)

        with torch.no_grad():
            for batch in data_loader:
                inputs, targets = batch
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)

                logits = self.model(inputs)
                metrics.update(logits, targets)

        results = metrics.compute()
        self._print_metrics(results)
        return results

    def evaluate_generation(self, prompts, max_length=100, temperature=0.7, top_k=40):
        """Generate responses for a set of prompts and analyze them"""
        generations = []

        for prompt in prompts:
            generated = generate(self.model, prompt, self.tokenizer,
                                max_length=max_length,
                                temperature=temperature,
                                top_k=top_k,
                                device=self.device)
            generations.append({
                'prompt': prompt,
                'generation': generated,
                'length': len(generated.split())
            })

        # Calculate generation statistics
        avg_length = sum(g['length'] for g in generations) / len(generations)

        print(f"===== Generation Evaluation =====")
        print(f"Temperature: {temperature}, Top-k: {top_k}")
        print(f"Average generation length: {avg_length:.2f} words")
        print("\nSample generations:")
        for i, gen in enumerate(generations[:3]):  # Show first 3 examples
            print(f"\nPrompt {i+1}: {gen['prompt']}")
            print(f"Generated: {gen['generation']}")

        return generations

    def evaluate_repetition(self, prompts, max_length=100, temperature=0.7):
        """Evaluate likelihood of repetitive text generation"""
        def get_ngram_counts(text, n):
            tokens = text.split()
            ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
            return Counter(ngrams)

        results = []
        for prompt in prompts:
            generated = generate(self.model, prompt, self.tokenizer,
                               max_length=max_length,
                               temperature=temperature,
                               device=self.device)

            # Only analyze the generated portion (excluding prompt)
            gen_text = generated[len(prompt):]

            # Calculate 3-gram and 4-gram repetition rates
            trigram_counts = get_ngram_counts(gen_text, 3)
            fourgram_counts = get_ngram_counts(gen_text, 4)

            # Repetition metrics
            total_trigrams = sum(trigram_counts.values())
            total_fourgrams = sum(fourgram_counts.values())

            unique_trigrams = len(trigram_counts)
            unique_fourgrams = len(fourgram_counts)

            # Repetition rate (lower is better - more unique n-grams)
            trigram_repetition = 1.0 - (unique_trigrams / total_trigrams) if total_trigrams > 0 else 0
            fourgram_repetition = 1.0 - (unique_fourgrams / total_fourgrams) if total_fourgrams > 0 else 0

            results.append({
                'prompt': prompt,
                'generation': gen_text,
                'trigram_repetition': trigram_repetition,
                'fourgram_repetition': fourgram_repetition
            })

        # Calculate averages
        avg_trigram = sum(r['trigram_repetition'] for r in results) / len(results)
        avg_fourgram = sum(r['fourgram_repetition'] for r in results) / len(results)

        print(f"===== Repetition Analysis =====")
        print(f"Average 3-gram repetition rate: {avg_trigram:.4f} (lower is better)")
        print(f"Average 4-gram repetition rate: {avg_fourgram:.4f} (lower is better)")

        return results

    def _print_metrics(self, metrics):
        """Print formatted metrics"""
        print("\n===== Model Performance =====")
        print(f"Perplexity: {metrics['perplexity']:.2f}")
        print(f"Loss:       {metrics['loss']:.4f}")
        print(f"Top-1 Acc:  {metrics['top1_acc']:.4f}")
        print(f"Top-5 Acc:  {metrics['top5_acc']:.4f}")

    def analyze_attention_patterns(self, prompt, layer_indices=None, head_indices=None):
        """
        Analyze attention patterns for specific layers and heads
        Requires capturing attention weights during forward pass
        """
        # Implement if your model can access attention weights
        # This would require model modification to return attention weights
        raise NotImplementedError("Attention analysis requires model hooks")

    def evaluate_temperature_sweep(self, prompt, temps=[0.5, 0.7, 0.9, 1.1, 1.3]):
        """Generate with different temperatures to analyze effect"""
        print("\n===== Temperature Sensitivity Analysis =====")
        print(f"Prompt: {prompt}")

        results = []
        for temp in temps:
            generated = generate(self.model, prompt, self.tokenizer,
                              temperature=temp,
                              max_length=50,
                              device=self.device)

            results.append({
                'temperature': temp,
                'generation': generated
            })

            print(f"\nTemperature {temp}:")
            print(generated)

        return results

In [ ]:
# Create validation data loader
val_ds = raw_ds.select(range(5000, 6000))  # Select a validation subset
val_tok_ds = val_ds.map(preprocess_fn, remove_columns=["text"])
val_tok_ds.set_format("torch", columns=["input_ids", "labels"])
val_loader = DataLoader(val_tok_ds, batch_size=16, collate_fn=collate_fn)

# Test prompts for generation evaluation
test_prompts = [
    "Once upon a time there was a",
    "The little dog ran into the",
    "She looked at the sky and saw",
    "Tom was sad because his friend"
]

# Initialize evaluator with trained model
evaluator = LLMEvaluator(trained_model, tokenizer)

# Run comprehensive evaluation
print("\nRunning model evaluation...")
metrics = evaluator.compute_metrics(val_loader)
generations = evaluator.evaluate_generation(test_prompts)
repetition_results = evaluator.evaluate_repetition(test_prompts[:2])
temp_results = evaluator.evaluate_temperature_sweep(test_prompts[0])

# Output overall assessment
print("\n===== EVALUATION SUMMARY =====")
print(f"Perplexity: {metrics['perplexity']:.2f}")
print(f"Text quality: {'Good' if metrics['perplexity'] < 10 else 'Needs improvement'}")
print(f"Repetition tendency: {'Low' if repetition_results[0]['fourgram_repetition'] < 0.3 else 'High'}")
print(f"Temperature sensitivity: Check above outputs for coherence/creativity balance")

The code implements a comprehensive evaluation framework for language models that combines quantitative metrics and qualitative generation assessment. The `ValidationMetrics` class serves as a statistical accumulator that tracks performance across batches, maintaining running lists of losses and accuracy measurements. During the `update` method, it computes cross-entropy loss between predicted logits and target tokens without gradient calculation, then applies softmax to convert raw logits into probability distributions. It extracts the top-5 predicted tokens using `torch.topk()` and compares them against ground truth targets (which are augmented with an extra dimension via `unsqueeze(-1)`) to determine both top-1 and top-5 accuracy—whether the correct token appears in the model's most confident prediction or among its top five candidates. The `compute` method synthesizes these accumulated statistics into four key metrics: perplexity (calculated as $e^{\text{mean loss}}$, representing the model's uncertainty over the vocabulary), raw loss, and the two accuracy measures. The standalone `validate` function wraps this metrics collection process into a complete evaluation loop, temporarily disabling dropout with `model.eval()`, processing batches without gradient tracking, and restoring training mode afterward. Building on these foundations, the `LLMEvaluator` class provides a higher-level interface that combines statistical evaluation with generative assessment. Its `compute_metrics` method reuses the `ValidationMetrics` machinery for standard statistical evaluation, while `evaluate_generation` applies the model to a set of prompts to analyze fluency and coherence. For deeper quality assessment, `evaluate_repetition` implements an n-gram analysis that measures lexical diversity by calculating the ratio of unique n-grams to total n-grams—effectively detecting text degeneration patterns where the model produces repetitive phrases. The method extracts trigrams and fourgrams (sequences of three and four consecutive words) from generated text, counts their occurrences using Python's `Counter`, and calculates repetition rates where lower values indicate greater diversity. The evaluator also includes `evaluate_temperature_sweep`, which demonstrates how sampling temperature affects generation quality by producing outputs at different randomness levels (from 0.5 for focused, deterministic responses to 1.3 for creative but potentially incoherent text). The final section shows the evaluation framework in action by creating a validation dataset from the TinyStories corpus, defining diverse test prompts, initializing the evaluator with a trained model, and executing a comprehensive assessment that combines perplexity measurements, generation samples, repetition analysis, and temperature sensitivity testing. The evaluation concludes with an interpretable summary that translates technical metrics into practical judgments: categorizing text quality based on perplexity thresholds and repetition tendency based on fourgram diversity scores, providing an accessible assessment for non-technical stakeholders while preserving detailed metrics for model developers.



### Putting It All Together

1. **Model**: builds embeddings → N transformer blocks → output projection.
2. **Trainer**: handles mixed-precision training, gradient clipping, and logging.
3. **Generation**: uses autoregressive loop with KV caching and top-k sampling.
4. **Checkpointing**: lets you save/load training progress reliably.
5. **Validation**: gives you loss/perplexity and accuracy metrics to monitor generalization.



## SOTA Models, Strategies & Use Cases

In this section, we'll examine state-of-the-art models that represent different design philosophies and capabilities in the current LLM landscape. Understanding these leading models provides valuable insights into architectural decisions, training strategies, and use case optimization

### 1. Google Gemini: Multimodal Reasoning at Scale

**Architecture & Unique Features:**
- Built as a multimodal model from the ground up (not retrofitted)
- Uses a mixture-of-experts (MoE) architecture with conditional computation
- Optimized for complex reasoning and multi-turn conversation
- Specialized vision encoders with segmentation capabilities
- Estimated 1.5-2.5T parameters (Ultra version)

**Primary Use Case: Multimodal Question Answering**

Gemini excels at tasks that require understanding relationships between different modalities (text, images, audio) and reasoning about them collectively. Its architecture enables it to:

- Process multiple images with text in context (e.g., analyzing steps in a visual procedure)
- Perform visual reasoning tasks (e.g., "What's different between these two diagrams?")
- Generate text responses that reference specific visual elements
- Solve complex multimodal problems that require spatial and logical reasoning

**Code Example:**
```python
from google.generativeai import configure, generate_content
import PIL.Image

# Configure API
configure(api_key="YOUR_API_KEY")

# Load image
image = PIL.Image.open("diagram.jpg")

# Generate response with multimodal context
response = generate_content(
    model="gemini-pro-vision",
    contents=[
        "What components are labeled in this circuit diagram?",
        image
    ]
)
print(response.text)
```



### 2. Meta Llama 3: Open Foundation Models

**Architecture & Unique Features:**
- Full-parameter transformer architecture (no MoE)
- Strong context window (128K tokens)
- Grouped-query attention mechanism
- Optimized training recipe with high-quality data filtering
- 8B to 70B parameter versions available with open weights

**Primary Use Case: Open-Source Chat and Tool Integration**

Llama 3 demonstrates the power of open foundation models that can be specialized for various use cases. Its design prioritizes:

- Adaptability for fine-tuning across domains
- Strong reasoning with well-documented limitations
- Integration with tool-use frameworks (function calling)
- Support for local deployment with reasonable resource requirements

**Code Example:**
```python
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer
model_name = "meta-llama/Llama-3-8B-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Format prompt for chat
prompt = f"""<|system|>
You are a helpful AI assistant.
<|user|>
How would I implement binary search in Python?
<|assistant|>
"""

# Generate response
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    inputs["input_ids"],
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9
)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))
```



### 3. Google Gemma: Efficient Deployment and Specialization

**Architecture & Unique Features:**
- Distilled and optimized from Google's larger models
- Transformer-based with architectural optimizations for efficiency
- Designed for fine-tuning and quantization
- 2B and 7B parameter versions available
- Open weights with responsible AI license

**Primary Use Case: Code Generation and Specialized Domain Adaptation**

Gemma exemplifies the trend toward smaller yet capable models that can be efficiently deployed and specialized. Its strengths include:

- Strong code generation capabilities relative to its size
- Efficient fine-tuning for domain-specific applications
- Low resource requirements for deployment (runs on consumer hardware)
- Quantization-friendly design for edge deployment

**Code Example:**
```python
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load Gemma model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it")

# Create a text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate code with a specific prompt
code_prompt = """
Write a Python function to find the longest palindromic substring in a given string.
```python
"""

response = generator(
    code_prompt,
    max_new_tokens=400,
    do_sample=True,
    temperature=0.3,
    top_k=50,
    top_p=0.95
)

print(response[0]["generated_text"])
```



### Summary: Key Trends in SOTA Models

1. **Architectural Evolution**
   - Mixture-of-Experts for scaling parameter count without increasing inference cost
   - Specialized attention mechanisms (grouped-query, multi-query)
   - Optimized context windows for long-range dependencies

2. **Modality Integration**
   - Purpose-built multimodal architectures vs. adapter approaches
   - Specialized encoders for different modalities
   - Cross-modal reasoning capabilities

3. **Deployment Strategies**
   - Size-optimized models for specific deployment scenarios
   - Quantization and distillation for efficiency
   - Open weights enabling community innovation

The choice between these models depends on your specific requirements: Gemini for complex multimodal reasoning at scale, Llama for adaptable open-source deployment with strong general capabilities, and Gemma for efficient specialized applications with limited resources. All three demonstrate the rapid evolution of transformer-based architectures to address different points in the capability vs. efficiency tradeoff space.

## Citations

- Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., Kaiser, Ł., & Polosukhin, I. (2017). Attention Is All You Need. In Advances in Neural Information Processing Systems. (Introduces the Transformer architecture.)
- Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. In NAACL. (Masking and bidirectional representations.)
- Radford, A., Narasimhan, K., Salimans, T., & Sutskever, I. (2018). Improving Language Understanding by Generative Pre-Training. (Early GPT work on unsupervised pretraining.)
- Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J., Dhariwal, P., ... & Amodei, D. (2020). Language Models are Few-Shot Learners. (GPT-3; large-scale autoregressive models.)
- Dosovitskiy, A., Beyer, L., Kolesnikov, A., Weissenborn, D., Zhai, X., Unterthiner, T., ... & Houlsby, N. (2021). An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale. (Vision Transformer — for image transformer models.)
- Clark, K., Khandelwal, U., Levy, O., & Manning, C. D. (2019). What Does BERT Look At? An Analysis of BERT's Attention. (Explainability and attention analysis.)
- Jain, S., & Wallace, B. C. (2019). Attention is not Explanation. (Caveats on interpreting attention as explanation.)
- Lin, Y., Li, S., & Li, Y. (2020). A Survey of Transformer Interpretability Methods. (Survey paper covering explanation techniques.)
- Pennington, J., Socher, R., & Manning, C. (2014). GloVe: Global Vectors for Word Representation. (Classic word embeddings.)
- Mikolov, T., Sutskever, I., Chen, K., Corrado, G. S., & Dean, J. (2013). Distributed Representations of Words and Phrases and their Compositionality. (Word2Vec.)

Additional resources and tutorials:

- The Illustrated Transformer — http://jalammar.github.io/illustrated-transformer/ (visual walkthrough of transformers and attention)
- Stanford CS224n lecture notes and assignments (excellent practical material on NLP and transformers)
- Hugging Face Transformers library — https://huggingface.co/docs/transformers (practical tools and models)


<a href="https://somwrks.notion.site/?source=copy_link" class="btn btn-primary btn-lg" style="background-color: #0366d6; color: white; padding: 5px 10px; border-radius: 5px; text-decoration: none; font-weight: bold; display: inline-block; margin-top: 10px;"><i class="fa fa-file-text-o" aria-hidden="true"></i> Research paper breakdowns</a> <a href="https://github.com/ashworks1706/rlhf-from-scratch" class="btn btn-primary btn-lg" style="background-color: #0366d6; color: white; padding: 5px 10px; border-radius: 5px; text-decoration: none; font-weight: bold; display: inline-block; margin-top: 10px;"><i class="fa fa-file-text-o" aria-hidden="true"></i> RLHF From Scratch</a> <a href="https://github.com/ashworks1706/llm-from-scratch" class="btn btn-primary btn-lg" style="background-color: #0366d6; color: white; padding: 5px 10px; border-radius: 5px; text-decoration: none; font-weight: bold; display: inline-block; margin-top: 10px;"><i class="fa fa-file-text-o" aria-hidden="true"></i> LLM From Scratch</a> <a href="https://github.com/ashworks1706/agents-rag-from-scratch" class="btn btn-primary btn-lg" style="background-color: #0366d6; color: white; padding: 5px 10px; border-radius: 5px; text-decoration: none; font-weight: bold; display: inline-block; margin-top: 10px;"><i class="fa fa-file-text-o" aria-hidden="true"></i> Agents & RAG From Scratch</a>